<a href="https://colab.research.google.com/github/totalleecher/codemaster-mm/blob/master/All_in_One_3_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/totalleecher/Codemaster">
<img src="https://github.com/fluidicon.png" height=100px, align="left"></a>



<center><h1><b>Welcome to My Page</h1></b><h5>You can remote upload and download torrent to your <b>Google Drive</b> Here</center></h5>



In [ ]:
#@markdown <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png" height=30px>      ___Mount / Unmount Gdrive!___
#@markdown <center><h6>Mount Gdrive to /content/drive</h6></center>



MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

In [ ]:
#@title ← Check Your Notebook Info (Optional)
#@markdown - Your CPU total threads should be <b>40</b> in <b>TPU Mode</b>, otherwise process may slow.
from psutil import *
!nvidia-smi
!python -V
#!cat /proc/version
!lscpu |grep 'Model name'
#!cat /proc/cpuinfo
!lscpu | grep 'Thread(s) per core'
!lscpu | grep "L3 cache"
!lscpu | grep 'Socket(s):'
!lscpu | grep 'Core(s) per socket:'
!lscpu | grep "MHz"
print("Total No of Thread(s) " + str(cpu_count()))

!cat /proc/meminfo | grep 'MemAvailable'
virtual_memory()
!df -h
!df -h / | awk '{print $4}'

# <center><b>Direct Link, Spotify, You Tube Link, Yandex, Google Drive Downloaders</b><h5>All of hidden cells are direct downloaders</center></h5>

## <img src='https://raw.githubusercontent.com/totalleecher/Google-Colab-CloudTorrent/master/src/aria2cli.png' height="30" alt="aira2"/> <font size=1px >___Direct link downloader / Remote Uploading___</font>
___Auto share link supported : Disk.yandex.com, Gdrive and youtube-dl all extractor...___

In [ ]:
%cd /content/drive/My Drive
%mkdir DirectDownload

#

URL = "https://gd.harrypro.workers.dev/1:/MM%20Channel/Movies/Adult/Be%20My%20Slave%20(Director's%20Cut).2012.BDRip.576p.x264.AAC.mp4-muxed.mp4" #@param {type:"string"}
#@markdown <center><h5>Default output path <b>/content/drive/My Drive/DirectDownload</b></h5></center>
OUTPUT_PATH = "/content" #@param {type:"string"}

import pathlib
import shutil
import hashlib
import requests
from urllib.parse import urlparse
from os import path, mkdir
if not path.exists("/root/.ipython/ttmg.py"): 
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/totalleecher/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py \
                    -O /root/.ipython/ttmg.py"
    run(split(shellCmd))
from ttmg import updateCheck, runSh

def youtubedlInstall():
  if not path.isfile("/usr/local/bin/youtube-dl"):
    cmdC = "rm -rf /content/sample_data/ && " \
            " mkdir -p -m 666 /root/.YouTube-DL/ &&" \
            " apt-get install atomicparsley &&" \
            " curl -L https://yt-dl.org/downloads/latest/youtube-dl " \
            "-o /usr/local/bin/youtube-dl &&" \
            " chmod a+rx /usr/local/bin/youtube-dl"
    get_ipython().system_raw(cmdC)

def aria2Install():
  runSh('apt install -y aria2')

def istmd(URL): 
  link = urlparse(URL)
    
  #YandexDisk
  if link.netloc == "yadi.sk":
    API_ENDPOINT = 'https://cloud-api.yandex.net/v1/disk/public/resources/' \
                    '?public_key={}&path=/{}&offset={}'
    dry = False
    def md5sum(file_path):
        md5 = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
                md5.update(chunk)
        return md5.hexdigest()


    def check_and_download_file(target_path, url, size, checksum):
        if path.isfile(target_path):
            if size == path.getsize(target_path):
                if checksum == md5sum(target_path):
                    print('URL {}'.format(url))
                    print('skipping correct {}'.format(target_path))
                    return
        if not dry:
            print('URL {}'.format(url))
            print('downloading {}'.format(target_path))
            runSh(f'aria2c -x 16 -s 16 -k 1M -d {OUTPUT_PATH} {url}', output=True)
            # r = requests.get(url, stream=True)
            # with open(target_path, 'wb') as f:
            #     shutil.copyfileobj(r.raw, f)


    def download_path(target_path, public_key, source_path, offset=0):
        print('getting "{}" at offset {}'.format(source_path, offset))
        current_path = path.join(target_path, source_path)
        pathlib.Path(current_path).mkdir(parents=True, exist_ok=True)
        jsn = requests.get(API_ENDPOINT.format(public_key, source_path, offset)).json()
        def try_as_file(j):
            if 'file' in j:
                file_save_path = path.join(current_path, j['name'])
                check_and_download_file(file_save_path, j['file'], j['size'], j['md5'])
                return True
            return False

        # first try to treat the actual json as a single file description
        if try_as_file(jsn):
            return

        # otherwise treat it as a directory
        emb = jsn['_embedded']
        items = emb['items']
        for i in items:
            # each item can be a file...
            if try_as_file(i):
                continue
            # ... or a directory
            else:
                subdir_path = path.join(source_path, i['name'])
                download_path(target_path, public_key, subdir_path)

        # check if current directory has more items
        last = offset + emb['limit']
        if last < emb['total']:
            download_path(target_path, public_key, source_path, last)
    download_path(OUTPUT_PATH, URL, '')
    return False  
  return URL

if not OUTPUT_PATH:
  OUTPUT_PATH = "/content/drive/My Drive/DirectDownload"
  
if not URL == "":
  aria2Install()
  youtubedlInstall()
  try:
    mkdir("DirectDownload")
  except FileExistsError:
    pass
  url = istmd(URL)
  if url != False:
    print('URL {}'.format(URL))
    cmdC = f'youtube-dl -o "{OUTPUT_PATH}/%(title)s" {URL} ' \
            '--external-downloader aria2c ' \
            '--external-downloader-args "-x 16 -s 16 -k 1M"'
    runSh(cmdC, output=True)
else:
  print("Please input url")

##<img src="https://ar.toneden.io/13766685/6070f58d-091b-4a80-bd5c-24378e308116" height="30" alt="Zippyshare logo"/>
___Remote upload from Zippyshare to Google drive___


In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
##@markdown <center><img src="https://ar.toneden.io/13766685/6070f58d-091b-4a80-bd5c-24378e308116" height="100" alt="Zippyshare logo"/></center>
##@markdown <center><h3>Remote upload from Zippyshare to Google drive</h3></center><br>



from IPython.display import HTML, clear_output

%cd /content
!git clone https://github.com/mcrapet/plowshare.git
%cd plowshare
!make install
!plowmod --install
!apt-get install nodejs
!cd ..


clear_output()

print("Server Started Successfully")


In [ ]:
#@markdown <h3>⬅️ Click Here to START Download</h3>

import os
direct_URL = "https://www18.zippyshare.com/v/74wtly9z/file.html" #@param {type:"string"}
output_dir = "/content/drive/My Drive" #@param {type:"string"}

!plowdown "$direct_URL" -o "$output_dir"

%cd /content/

In [ ]:
#@markdown <h3>⬅️ Click Here to START Batch Download</h3>
#@markdown <h5> Make "download.txt" file contaning batch links of zippyshare and upload here to down them all.

%cd /content/plowshare

!rm *.txt

from google.colab import files
files.upload()

import os
output_dir = "/content/drive/My Drive" #@param {type:"string"}

!plowdown "download.txt" -m -o "$output_dir"

%cd /content/





##<left><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/57/01_mega_logo.svg/800px-01_mega_logo.svg.png" height="30" alt="Zippyshare logo"/></left> <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png" height=30px align="right">
___Remote upload from Mega, Gdrive to Google drive___

In [ ]:
#@markdown <center><h5>Easy to use but we recommend you to use <font color="red"><b>Jdownloader</b></font> when download from mega.co.nz</h5></center><br>
!pip install mega.py
!pip install gdget

##
import os
from IPython.display import HTML, clear_output
%cd /content/drive/My Drive

Gdrive_url = "" #@param {type:"string"}
MegaNZ_url = "https://mega.nz/file/l8MyzZba#P8WB63QZ83T_opN8TaGAGku43rx2mo9OsplP7j3q4fk" #@param {type:"string"}
#Direct_Link = "" #@param {type:"string"}
Download_from = "MegaNZ" #@param ["MegaNZ", "Gdrive"]
clear_output()
print('Downloading started')

if Download_from == 'MegaNZ':
 os.makedirs("Mega", exist_ok=True)
 %cd Mega
 from mega import Mega
 mega = Mega()
 m = mega.login()
 m.download_url(MegaNZ_url)
 print('Finished')

if Download_from == 'Gdrive':
 os.makedirs("Gdrive", exist_ok=True)
 %cd Gdrive
 !gdget "$Gdrive_url"
 print('Finished')



## <img src='https://biplobsd.github.io/RLabClone/img/title_jdownloader.png' height="30" alt="JDownloader"/>

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Execute JDownloader
# @markdown Example : totalleecher@gmail.com
NEW_Account = False  # @param {type:"boolean"}
# ================================================================ #

from os import path as _p

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))  # nosec

from rlab_utils import handleJDLogin

handleJDLogin(NEW_Account)

## <img src='https://biplobsd.github.io/RLabClone/img/title_youtube-dl.png' height="30" alt="Youtube-DL"/>

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Execute YouTube-DL
# @markdown 📝 Note: if you want to change an archive file just run this cell again.
Archive = False #@param {type:"boolean"}
# ================================================================ #

import os, uuid, urllib.parse
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files
os.system('sudo pip install youtube_dl')
Links = widgets.Textarea(placeholder='''Video/Playlist Link
(one link per line)''')

VideoQ = widgets.Dropdown(options=["Best Quality (VP9 upto 4K)", "Best Compatibility (H.264 upto 1080p)"])

AudioQ = widgets.Dropdown(options=["Best Quality (Opus)", "Best Compatibility (M4A)"])

Subtitle = widgets.ToggleButton(value=True, description="Subtitle", button_style="info", tooltip="Subtitle")

SavePathYT = widgets.Dropdown(options=["/content", "/content/Downloads"])

AudioOnly = widgets.ToggleButton(value=False, description="Audio Only", button_style="", tooltip="Audio Only")

Resolution = widgets.Select(options=["Highest", "4K", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"], value="Highest")

Extension = widgets.Select(options=["mkv", "webm"], value="mkv")

UsernameYT = widgets.Text(placeholder="Username")

PasswordYT = widgets.Text(placeholder="Password")

SecAuth = widgets.Text(placeholder="2nd Factor Authentication")

VideoPW = widgets.Text(placeholder="Video Password")

GEOBypass = widgets.Dropdown(options=["Disable", "Hide", "AD", "AE", "AF", "AG", "AI", "AL", "AM", "AO", "AQ", "AR", "AS", "AT", "AU", "AW", "AX", "AZ", "BA", "BB", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BL", "BM", "BN", "BO", "BQ", "BR", "BS", "BT", "BV", "BW", "BY", "BZ", "CA", "CC", "CD", "CF", "CG", "CH", "CI", "CK", "CL", "CM", "CN", "CO", "CR", "CU", "CV", "CW", "CX", "CY", "CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EE", "EG", "EH", "ER", "ES", "ET", "FI", "FJ", "FK", "FM", "FO", "FR", "GA", "GB", "GD", "GE", "GF", "GG", "GH", "GI", "GL", "GM", "GN", "GP", "GQ", "GR", "GS", "GT", "GU", "GW", "GY", "HK", "HM", "HN", "HR", "HT", "HU", "ID", "IE", "IL", "IM", "IN", "IO", "IQ", "IR", "IS", "IT", "JE", "JM", "JO", "JP", "KE", "KG", "KH", "KI", "KM", "KN", "KP", "KR", "KW", "KY", "KZ", "LA", "LB", "LC", "LI", "LK", "LR", "LS", "LT", "LU", "LV", "LY", "MA", "MC", "MD", "ME", "MF", "MG", "MH", "MK", "ML", "MM", "MN", "MO", "MP", "MQ", "MR", "MS", "MT", "MU", "MV", "MW", "MX", "MY", "MZ", "NA", "NC", "NE", "NF", "NG", "NI", "NL", "NO", "NP", "NR", "NU", "NZ", "OM", "PA", "PE", "PF", "PG", "PH", "PK", "PL", "PM", "PN", "PR", "PS", "PT", "PW", "PY", "QA", "RE", "RO", "RS", "RU", "RW", "SA", "SB", "SC", "SD", "SE", "SG", "SH", "SI", "SJ", "SK", "SL", "SM", "SN", "SO", "SR", "SS", "ST", "SV", "SX", "SY", "SZ", "TC", "TD", "TF", "TG", "TH", "TJ", "TK", "TL", "TM", "TN", "TO", "TR", "TT", "TV", "TW", "TZ", "UA", "UG", "UM", "US", "UY", "UZ", "VA", "VC", "VE", "VG", "VI", "VN", "VU", "WF", "WS", "YE", "YT", "ZA", "ZM", "ZW"])

ProxyYT = widgets.Text(placeholder="Proxy URL")

MinSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Min:")

MaxSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Max:")

ExtraArg = widgets.Text(placeholder="Extra Arguments")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def upload_archive():
  if ask_yes_no("Do you already have an archive file? (y/n)", default="", interrupt=""):
    try:
      display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Please upload an archive from your computer.</h2><br>"))
      UploadConfig = files.upload().keys()
      clear_output(wait=True)
      if len(UploadConfig) == 0:
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
      elif len(UploadConfig) == 1:
        for fn in UploadConfig:
          if os.path.isfile("/content/" + fn):
            get_ipython().system_raw("mv -f " + "\"" + fn + "\" /root/.youtube-dl.txt && chmod 666 /root/.youtube-dl.txt")
            AudioOnly.observe(AudioOnlyChange)
            Subtitle.observe(SubtitleChange)
            AudioQ.observe(AudioQChange)
            ShowYT()
          else:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
      else:
        for fn in UploadConfig:
          get_ipython().system_raw("rm -f " + "\"" + fn + "\"")
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
    except:
      clear_output(wait=True)
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))
  else:
    get_ipython().system_raw("touch '/root/.youtube-dl.txt'")
    AudioOnly.observe(AudioOnlyChange)
    Subtitle.observe(SubtitleChange)
    AudioQ.observe(AudioQChange)
    ShowYT()

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content", "/content/Downloads"]

def AudioOnlyChange(change):
  if change["type"] == "change" and change["new"]:
    VideoQ.disabled = True
    Subtitle.disabled = True
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = True
    Extension.options = ["best", "aac", "flac", "mp3", "m4a", "opus", "vorbis", "wav"]
    Extension.value = "best"
    AudioOnly.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    VideoQ.disabled = False
    Subtitle.disabled = False
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = False
    if AudioQ.value == "Best Quality (Opus)":
      Extension.options = ["mkv", "webm"]
    else:
      Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"
    AudioOnly.button_style = ""

def SubtitleChange(change):
  if change["type"] == "change" and change["new"]:
    Subtitle.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    Subtitle.button_style = ""

def AudioQChange(change):
  if change["type"] == "change" and change["new"] == "Best Quality (Opus)":
    Extension.options = ["mkv", "webm"]
    Extension.value = "mkv"
  elif change["type"] == "change" and change["new"] == "Best Compatibility (M4A)":
    Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"

def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  display(widgets.HBox([widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                      widgets.HTML("<b style=\"color:#888888;\">For website that require an account:</b>"), UsernameYT, PasswordYT, SecAuth, VideoPW,
                                      widgets.HTML("<b><a href=\"https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Officially_assigned_code_elements\" target=\"_blank\">GEO Bypass Country:</a></b>"), GEOBypass,
                                      widgets.HTML("<b style=\"color:#888888;\">Proxy:</b>"), ProxyYT,
                                      widgets.HTML("<b style=\"color:#888888;\">Sleep Interval (second):</b>"), MinSleep, MaxSleep]),
                        widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Video Quality:</b>"), VideoQ, widgets.HTML("<b style=\"color:#888888;\">Resolution:</b>"), Resolution,
                                      widgets.HTML("<b style=\"color:#888888;\">Audio Quality:</b>"), AudioQ, widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Options:</b>"), widgets.HBox([Subtitle, AudioOnly]),
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), ExtraArg])]), HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"),
          SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    # Account Check
    if UsernameYT.value.strip() and PasswordYT.value.strip():
      accountC = "--username \"" + UsernameYT.value + "\" --password \"" + PasswordYT.value + "\""
    else:
      accountC = ""
    if SecAuth.value.strip():
      secauthC = "-2 " + SecAuth.value
    else:
      secauthC = ""
    if VideoPW.value.strip():
      videopwC = "--video-password " + VideoPW.value
    else:
      videopwC = ""
    # Proxy
    if ProxyYT.value.strip():
      proxyytC = "--proxy " + ProxyYT.value
    else:
      proxyytC = ""
    # GEO Bypass
    if GEOBypass.value == "Disable":
      geobypass = ""
    elif GEOBypass.value == "Hide":
      geobypass = "--geo-bypass"
    else:
      geobypass = "--geo-bypass-country " + GEOBypass.value
    # Video Quality
    if VideoQ.value == "Best Quality (VP9 upto 4K)":
      videoqC = "webm"
    else:
      videoqC = "mp4"
    # Audio Quality
    if AudioQ.value == "Best Quality (Opus)":
      audioqC = "webm"
    else:
      audioqC = "m4a"
    # Audio Only Check
    if AudioOnly.value:
      subtitleC = ""
      thumbnailC = ""
      extC = "-x --audio-quality 0 --audio-format " + Extension.value
      codecC = "bestaudio[ext=" + audioqC + "]/bestaudio/best"
    else:
      if Subtitle.value:
        subtitleC = "--all-subs --convert-subs srt --embed-subs"
      else:
        subtitleC = ""
      if Extension.value == "mp4":
        thumbnailC = "--embed-thumbnail"
      else:
        thumbnailC = ""
      extC = "--merge-output-format " + Extension.value
      if Resolution.value == "Highest":
        codecC = "bestvideo[ext=" + videoqC + "]+bestaudio[ext=" + audioqC + "]/bestvideo+bestaudio/best"
      else:
        codecC = "bestvideo[ext=" + videoqC + ",height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio[ext=" + audioqC + "]/bestvideo[height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio/bestvideo+bestaudio/best"
    # Archive
    if os.path.isfile("/root/.youtube-dl.txt"):
      archiveC = "--download-archive \"/root/.youtube-dl.txt\""
    else:
      archiveC = ""
    # Sleep Interval
    if MinSleep.value > 0 and MaxSleep.value > 0:
      minsleepC = "--min-sleep-interval " + MinSleep.value
      maxsleepC = "--max-sleep-interval " + MaxSleep.value
    else:
      minsleepC = ""
      maxsleepC = ""
    # Extra Arguments
    extraargC = ExtraArg.value
    for Link in Links.value.splitlines():
      clear_output(wait=True)
      Count += 1
      display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Processing link " + str(Count) + " out of " + Total + "</h3>"))
      if "youtube.com" in Link or "youtu.be" in Link:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading...</h3><br>"), YouTubeVideo(Link, width=640, height=360), HTML("<br>"))
      else:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading <a href=\"" + Link + "\">" + Link + "</a></h3><br>"))
      if ("youtube.com" in Link or "youtu.be" in Link) and "list=" in Link:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(playlist)s/%(playlist_index)s - %(title)s.%(ext)s" "$Link"
      else:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(title)s.%(ext)s" "$Link"
      if not os.path.exists(SavePathYT.value):
        get_ipython().system_raw("mkdir -p -m 666 " + SavePathYT.value)
      get_ipython().system_raw("mv /root/.YouTube-DL/* '" + SavePathYT.value + "/'")
    # Archive Download
    if os.path.isfile("/root/.youtube-dl.txt"):
      files.download("/root/.youtube-dl.txt")
    ShowYT()

if not os.path.isfile("/usr/local/bin/youtube-dl"):
  get_ipython().system_raw("rm -rf /content/sample_data/ && mkdir -p -m 666 /root/.YouTube-DL/ && apt-get install atomicparsley && curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl && chmod a+rx /usr/local/bin/youtube-dl")
if Archive:
  upload_archive()
else:
  AudioOnly.observe(AudioOnlyChange)
  Subtitle.observe(SubtitleChange)
  AudioQ.observe(AudioQChange)
  ShowYT()

##URL Hash
___Convert direct HTTP link to .torrent___

In [ ]:
import os, sys, re
#@markdown <h3>Please input url to make a torrent file</h3>

!cd /content/

#Install mktorrent
os.system('git clone https://github.com/pobrn/mktorrent.git')
%cd /content/mktorrent
os.system('make install')



url =  "" #@param {type:"string"}

if url != "":     
#Download file
 !wget "$url"

#Getting file name
 filename = url.split("/")[-1]
 split_string = filename.split(".", 1)
 tfilename = split_string[0]
 print(tfilename)
 print(filename)
 print(url)
      


#Create torrent using mktorrent
 !mktorrent -o "$tfilename".torrent -w "$url" "$filename"

 torrent = (tfilename)+"."+"torrent"

 from google.colab import files
 files.download(torrent) 

else:
 print("Please input url")

#Beta version V1.0 by Codemaster

##<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/Pornhub-logo.svg/512px-Pornhub-logo.svg.png' height="30" alt="filebrowser" />  <h5>

In [ ]:
#@markdown <h3>⬅️ Click Here to Install Require Modules</h3>
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/totalleecher/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip3 install youtube-dl')
os.system('pip3 install prettytable')
os.system('pip3 install bs4')
os.system('pip3 install requests')
%cd /content
os.system('git clone https://github.com/mariosemes/PornHub-downloader-python.git')

clear_output()
print("Install Finished")

In [ ]:
#@markdown <h3>⬅️ Click Here to download video</h3>

#@markdown - Single Download link Eg: https://www.pornhub.com/view_video.php?viewkey=ph5d69a2093729e
#@markdown - The batch option will ask you for the full path of your .txt file where you can import multiple URLs at once.Take care that every single URL in the .txt file is in his own row.



pornhub_url = '' #@param {type: "string"}
option = "single_download" #@param ["single_download", "batch_download","add","delete"]

%cd PornHub-downloader-python

if option == 'single_download':
  !python3 phdler.py custom "$pornhub_url"

elif option == 'add':
  !python3 phdler.py add "$pornhub_url"

elif option == 'delete':
  !python3 phdler.py delete "$pornhub_url"

else:
  !python3 phdler.py custom batch 

## <img src='https://github.com/pyload/pyload-webui/raw/master/media/banner.png' height="30" alt="pyLoad" /> <font size=1px>___PyLoad___</font>

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
##@markdown <br><center><img src='https://github.com/pyload/pyload-webui/raw/master/media/banner.png' height="100" alt="pyLoad"/></center>
##@markdown <center><h3>pyLoad Webinterface<br />The free and open-source Download Manager written in pure Python</h3></center><br>
import os, pathlib, zipfile, re, threading
import urllib.request
from IPython.display import HTML, clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "SA" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
#@markdown Default <br>&emsp;username : admin<br>&emsp;password : admin
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/totalleecher/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess
)

loadingAn()

os.makedirs('downloads', exist_ok=True)
os.makedirs('tools/pyload', exist_ok=True)

# Downloading latest version of pyload
if not os.path.exists("tools/pyload/pyload-stable"):
  urlF = 'https://github.com/pyload/pyload/archive/stable.zip'
  conf = 'https://raw.githubusercontent.com/totalleecher/' \
          'OneClickRun/master/res/appdata/pyload/pyload.conf'
  db = 'https://github.com/totalleecher/OneClickRun/raw/master/' \
        'res/appdata/pyload/files.db'
  urllib.request.urlretrieve(urlF, 'tools/pyload.zip')
  urllib.request.urlretrieve(conf, 'tools/pyload/pyload.conf')
  urllib.request.urlretrieve(db, 'tools/pyload/files.db')
  with zipfile.ZipFile('tools/pyload.zip', 'r') as zip_ref:
      zip_ref.extractall('tools/pyload')
  try:
    pathlib.Path('tools/pyload.zip').unlink()
  except FileNotFoundError:
    pass

  # runSh("apt install -y python-pycurl python-qt4 tesseract-ocr libtesseract-dev")
  # runSh("pip2 install pycrypto pyOpenSSL Jinja2 tesseract tesseract-ocr")

  # Using Threading to install
  threadRun = [
    threading.Thread(target=runSh, args=("apt install -y python-pycurl python-qt4 tesseract-ocr libtesseract-dev",)),
    threading.Thread(target=runSh, args=("pip2 install pycrypto pyOpenSSL Jinja2 tesseract tesseract-ocr",))
  ]
  for perTh in threadRun:perTh.start()
  for perTh in threadRun:perTh.join()

if not findProcess("python2.7", "pyLoadCore.py"):
  runCmd = "python2.7 /content/tools/pyload/pyload-stable/pyLoadCore.py" \
          " --configdir=/content/tools/pyload" \
          " --no-remote" \
          " --daemon"
  runSh(runCmd, shell=True)
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rcloneConfig', 53682, 'http'], 
                            ['pyload', 8000, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/rclonePyload.yml", 4074]
).start('pyload')

## <img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/spotify-downloader.png' height="30" alt="spotify-downloader" /> ___spotify-downloader___

In [ ]:
##@markdown <br><center><img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/spotify-downloader.png' height="100" alt="spotify-downloader"/></center>
#@markdown <center><h7>Download Spotify playlists from YouTube with albumart and meta-tags</h7></center><br>

import os, uuid, urllib.parse, re
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files


os.makedirs('tools/spotify-downloader/', exist_ok=True)
os.makedirs('downloads', exist_ok=True)
LATEST_VERSION = False #@param {type:"boolean"}

SET_VERSION = "2.0.8" #@param {type:"string"}
#@markdown  <font size='2px'>`https://pypi.org/project/spotdl/#history`</font>
# # Config files
# data = """spotify-downloader:
#   avconv: false
#   download-only-metadata: false
#   dry-run: false
#   file-format: '{artist} - {track_name}'
#   folder: /home/user/Music
#   input-ext: .m4a
#   log-level: INFO
#   manual: false
#   music-videos-only: false
#   no-fallback-metadata: false
#   no-metadata: false
#   no-spaces: false
#   output-ext: .mp3
#   overwrite: prompt
#   search-format: '{artist} - {track_name} lyrics'
#   skip: null
#   spotify_client_id: 4fe3fecfe5334023a1472516cc99d805
#   spotify_client_secret: 0f02b7c483c04257984695007a4a8d5c
#   trim-silence: false
#   write-successful: null
#   write-to: null
#   youtube-api-key: null
# """
# with open('tools/spotify-downloader/config.yml', 'w') as wnow:
#   wnow.write(data)

Links = widgets.Textarea(placeholder='''Link list
(one link per line)''')

fileFormat = widgets.Text(
    value='{artist} - {track_name}',
    placeholder='File name format',
    description="""File Name : file format to save the downloaded track with, each
                        tag is surrounded by curly braces. Possible formats:
                        ['track_name', 'artist', 'album', 'album_artist',
                        'genre', 'disc_number', 'duration', 'year',
                        'original_date', 'track_number', 'total_tracks',
                        'isrc']""",
    disabled=False
)

searchFormat = widgets.Text(
    value='{artist} - {track_name} lyrics',
    placeholder='Search format',
    description="""Search Format : search format to search for on YouTube, each tag is
                        surrounded by curly braces. Possible formats:
                        ['track_name', 'artist', 'album', 'album_artist',
                        'genre', 'disc_number', 'duration', 'year',
                        'original_date', 'track_number', 'total_tracks',
                        'isrc']""",
    disabled=False
)

tab = widgets.Tab()

LinksType = widgets.RadioButtons(
    options=['Songs', 'Playlist', 'Album', 'Username', 'Artist'],
    value='Songs',
    layout={'width': 'max-content'},
    description='Links type:',
    disabled=False,
)

SavePathYT = widgets.Dropdown(options=["/content/downloads", "/content"])

Extension = widgets.Select(options=["flac", "mp3", "m4a"], value="mp3")

TrimSilence = widgets.ToggleButton(
    value=False,
    description='Trim silence',
    disabled=False,
    button_style='',
    tooltip='remove silence from the start of the audio',
    icon='check'
)

writeM3u = widgets.ToggleButton(
    value=False,
    description='Write .m3u playlist',
    disabled=False,
    button_style='',
    tooltip='''generate an .m3u playlist file with youtube links
               given a text file containing tracks''',
    icon='check'
)

noMeta = widgets.ToggleButton(
    value=False,
    description='No metadata',
    disabled=False,
    button_style='',
    tooltip='do not embed metadata in tracks',
    icon='check'
)

noEnco = widgets.ToggleButton(
    value=False,
    description='No Encoding',
    disabled=False,
    button_style='',
    tooltip='do not encode media using FFmpeg',
    icon='check'
)

overwrite = widgets.Dropdown(
    disabled=False,
    options=['prompt', 'force', 'skip'],
    value='skip',
    description='Overwrite:',
)

quality = widgets.Dropdown(
    disabled=False,
    options=['worst', 'best'],
    value='best',
    description='Quality:',
)

inputExt = widgets.Dropdown(
    disabled=False,
    options=['automatic', 'm4a', 'opus'],
    value='automatic',
    description='Input format:',
)

logLevel = widgets.Dropdown(
    disabled=False,
    options=['INFO', 'WARNING', 'ERROR', 'DEBUG'],
    value='INFO',
    description='Log level:',
)

nf = widgets.ToggleButton(
    value=False,
    description='No fallback metadata',
    disabled=False,
    button_style='',
    tooltip='''do not use YouTube as fallback for metadata if track
                        not found on Spotify''',
    icon='check'
)

dryRun = widgets.ToggleButton(
    value=False,
    description='Dry run',
    disabled=False,
    button_style='',
    tooltip=''' show only track title and YouTube URL, and then skip
                        to the next track (if any)''',
    icon='check'
)

MusicVidOnly = widgets.ToggleButton(
    value=False,
    description='Music Videos Only',
    disabled=False,
    button_style='',
    tooltip='''search only for music videos on Youtube (works only
                        when YouTube API key is set''',
    icon='check'
)

NoSpaces = widgets.ToggleButton(
    value=False,
    description='No Spaces',
    disabled=False,
    button_style='',
    tooltip='''replace spaces with underscores in file names''',
    icon='check'
)

manual = widgets.ToggleButton(
    value=False,
    description='manually',
    disabled=False,
    button_style='',
    tooltip='''choose the track to download manually from a list of
                        matching tracks''',
    icon='check'
)

nr = widgets.ToggleButton(
    value=False,
    description='Keep original',
    disabled=False,
    button_style='',
    tooltip='''do not remove the original file after conversion''',
    icon='check'
)

ExtraArg = widgets.Text(placeholder="Extra Arguments")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content/downloads", "/content"]


def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  mainTab = widgets.Box([
                        widgets.HBox([widgets.VBox([
                                      widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                      LinksType, searchFormat, fileFormat, 
                                      widgets.HBox([TrimSilence, writeM3u, noMeta]),
                                      widgets.HBox([nf, dryRun, MusicVidOnly]),
                                      widgets.HBox([NoSpaces, manual, nr]),
                                      widgets.HBox([noEnco])
                                      ]),
                        widgets.VBox([
                                      widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), 
                                      Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), 
                                      ExtraArg, overwrite, quality, inputExt, 
                                      logLevel])
                        ])])
  
  tab.children = [mainTab]
  tab.set_title(0, 'spotify-downloader')
  display(HTML("<style>widget-tab-bar {display: none;}</style>"))
  display(tab)
  display(HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"), SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    
    if writeM3u.value:
      M3u = '--write-m3u'
    else:
      M3u = ''
    
    if TrimSilence.value:
      trmS = '--trim-silence'
    else:
      trmS = ''
    
    if noMeta.value:
      noM = '--no-metadata'
    else:
      noM = ''
    
    if noEnco.value:
      noE = '--no-encode'
    else:
      noE = ''

    if not overwrite.disabled:
      oW = f'--overwrite {overwrite.value}'
    else:
      oW = ''

    if not quality.disabled:
      qL = f'--quality {quality.value}'
    else:
      qL = ''
    
    if not inputExt.disabled:
      iE = f'--input-ext {inputExt.value}'
    else:
      iE = ''

    if not logLevel.disabled:
      lL = f'--log-level {logLevel.value}'
    else:
      lL = ''

    if nf.value:
      nfv = '--no-fallback-metadata'
    else:
      nfv = ''
    
    if dryRun.value:
      drR = '--dry-run'
    else:
      drR = ''
    
    if MusicVidOnly.value:
      MsV = '--music-videos-only'
    else:
      MsV = ''
    
    if NoSpaces.value:
      NoS = '--no-spaces'
    else:
      NoS = ''
    
    if manual.value:
      mal = '--manual'
    else:
      mal = ''
    
    if nr.value:
      nro = '--no-remove-original' 
    else:
      nro = ''
    
    if not searchFormat.value == '{artist} - {track_name} lyrics':
      seFor = f'--search-format "{searchFormat.value}"'
    else:
      seFor = ''
    
    if not fileFormat.value == '{artist} - {track_name}':
      fiFor = f'--file-format "{fileFormat.value}"'
    else:
      fiFor = ''
    
    if not LinksType.value == 'Songs':
      with open('tools/spotify-downloader/finish.txt', 'a+') as master:
        for Link in Links.value.splitlines():
          if LinksType.value == 'Playlist':
            outFileName = !spotdl --playlist $Link
          elif LinksType.value == 'Album':
            outFileName = !spotdl --album $Link
          elif LinksType.value == 'Username':
            outFileName = !spotdl -u $Link
          elif LinksType.value == 'Artist':
            outFileName = !spotdl --all-albums $Link
          filename = re.search(r"to\s(.+\.txt)", outFileName[-1]).group(1)
          with open(filename, 'r') as r:
            master.write(r.read())
    else:
      for Link in Links.value.splitlines():
        with open('tools/spotify-downloader/finish.txt', 'w') as master:
          master.write(Link)
    # Extra Arguments
    
    extraargC = ExtraArg.value
    cmd = r"spotdl -l 'tools/spotify-downloader/finish.txt' " \
      fr"-f '{SavePathYT.value}' " \
      fr"-o {Extension.value} " \
      f"{seFor} {fiFor} " \
      f"{M3u} {trmS} {oW} {noM} {noE} {nfv} {drR} {MsV} {NoS} {mal} {nro}" \
      f"{qL} {iE} {lL}" 
    !$cmd
    ShowYT()

if not os.path.isfile("/usr/local/bin/spotdl"):
  get_ipython().system_raw(f"pip3 install spotdl{'=='+SET_VERSION if LATEST_VERSION else ''} && apt -y install ffmpeg")

ShowYT()

## <font size=1px>___aria2___</font>

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
##@markdown <br><center><img src='...' height="200" alt="aria2"/></center>
#@markdown <center><h7>The next generation download utility.</h7></center><br>
import os, pathlib, zipfile, re
import urllib.request, requests
from IPython.display import HTML, clear_output
from urllib.parse import urlparse

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
REGION = "JP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
# Aria2 Service
Aria2_rpc = True  # @param {type:"boolean"}
Ariang_WEBUI = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
# OUTPUT_DIR = ""  # @param {type:"string"}
PORT = 8221
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/totalleecher/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    CWD,
    displayUrl,
    findProcess,
    findPackageR
)

loadingAn()

# Ariang SETUP
runSh('apt install -y aria2')
pathlib.Path('ariang').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)

# github latest releases tag define
def latestTag(link):
  import re
  from urllib.request import urlopen
  htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
  return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]

# Downloading latest version of ariang
if not os.path.exists("ariang/index.html"):
  # BASE_URL = r"https://github.com/mayswind/AriaNg"
  # LATEST_TAG = latestTag(BASE_URL)
  # urlF = f'{BASE_URL}/releases/download/{LATEST_TAG}/' \
  #             f'AriaNg-{LATEST_TAG}-AllInOne.zip'
  urllib.request.urlretrieve(
      findPackageR('mayswind/AriaNg', 'AllInOne.zip'),
      'ariang/new.zip')
  with zipfile.ZipFile('ariang/new.zip', 'r') as zip_ref:
      zip_ref.extractall('ariang')
  try:
    pathlib.Path('ariang/new.zip').unlink()
  except FileNotFoundError:
    pass

# START_ARIANG_WEBUI_AND_ARIA2_RPC
try:
  if not OUTPUT_DIR:
    OUTPUT_DIR = f"downloads/"
  elif not os.path.exists(OUTPUT_DIR):
    clear_output()
    print("Error: Your set path not found! Create path!")
    exx()
except:
  OUTPUT_DIR = f"{CWD}/downloads/"
if Aria2_rpc:
  if not findProcess("aria2c", "--enable-rpc"):
    try:
      trackers = requests.get(
          "https://trackerslist.com/best_aria2.txt"
      ).text
      cmdC = r"aria2c --enable-rpc --rpc-listen-port=6800 -D " \
            fr"-d {OUTPUT_DIR} " \
            r"-j 20 " \
            r"-c " \
            fr"--bt-tracker={trackers} " \
            r"--bt-request-peer-speed-limit=0 " \
            r"--bt-max-peers=0 " \
            r"--seed-ratio=0.0 " \
            r"--max-connection-per-server=10 " \
            r"--min-split-size=10M " \
            r"--follow-torrent=mem " \
            r"--disable-ipv6=true " \
            r"--user-agent=Transmission/2.77 --peer-agent=Transmission/2.77 " \
            r"--peer-id-prefix=-TR2770- " \
            r"--split=20 " \
            r" &"
      runSh(
          cmdC,
          shell=True
      )
    except:
      print("aria2 rpc not enabling. Try again later!")

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
if Aria2_rpc:
  Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Aria2_rpc', 6800, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/aria2.yml", 5042]
)
  data = Server.start('Aria2_rpc', displayB=False)
  Host = urlparse(data['url']).hostname
  port = "80"
  clear_output()
if Ariang_WEBUI:
  if Aria2_rpc:
    filePath = 'ariang/index.html'
    with open(filePath, 'r+') as f:
      read_data = f.read()
      f.seek(0)
      f.truncate(0)
      read_data = re.sub(r'(rpcHost:"\w+.")|rpcHost:""', 
                        f'rpcHost:"{Host}"', read_data)
      read_data = re.sub(r'protocol:"\w+."', r'protocol:"ws"', read_data)
      read_data = re.sub(r'rpcPort:"\d+."', f'rpcPort:"{port}"', read_data)
      f.write(read_data)
  try:
    urllib.request.urlopen(f"http://localhost:{PORT}")
  except:
    runSh(f"python3 -m http.server {PORT} &", shell=True, cd="ariang/")
  
  Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Ariang', PORT, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/ariang.yml", 5043]
)
  data2 = Server.start('Ariang', displayB=False)
  data2['url'] = urlparse(data2['url'])._replace(scheme='http').geturl()
  displayUrl(data2, pNamU='Ariang : ')

if Aria2_rpc:
  display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 100%;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>Aria2 rpc config<p><thead>        <tr>            <th>Protocol</th>            <th>Host</th>            <th>Port</th>        </tr>    </thead>    <tbody>        <tr>            <td>WebSocket</td><td>"""+Host+"""</td><td>"""+port+"""</td></tr></tbody></table><a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/aria2RpcConfigSetup.gif">NB. How to setup this's config. [Click ME]</a></button><center>"""))


#<center><b>Torrent Downloaders</b><h5>All of hidden cells are torrent downloaders</center></h5>

## <img src='https://colab.research.google.com/img/colab_favicon_256px.png' height="30" alt="colab" /> ___Torrent to Gdrive___



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to START server</h3>


!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

from IPython.display import HTML, clear_output

clear_output()

print("Server Started Successfully")



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Upload torrent file</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>



from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Add From Magnet Link</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>

params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to Start Download</h3>
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


## <img src='https://avatars1.githubusercontent.com/u/223312?s=400&v=4' height="30" alt="colab" />  __Transmission__



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to START server</h3>


!apt-get install transmission-cli
from IPython.display import HTML, clear_output
clear_output()
print("Server Started Successfully")



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Add From Magnet Link or .torrent Link</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>


import os
torrent_URL = "magnet:?xt=urn:btih:343fb15d0511f23047d7ceef1f7fae66f8828f88&dn=Risa+Kasumi+photos&tr=udp://tracker.openbittorrent.com:80&tr=udp://tracker.opentrackr.org:1337/announce" #@param {type:"string"}
output_dir = "/content/drive/My Drive" #@param {type:"string"}

!transmission-cli -w "$output_dir" "$torrent_URL"



## <img src='https://upload.wikimedia.org/wikipedia/commons/c/c5/Deluge_icon.png' height="30" alt="Deluge" /> __Deluge__

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
##@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/c/c5/Deluge_icon.png' height="200" alt="netdata"/></center>
#@markdown <center><h6>Deluge is a BitTorrent client written in Python.</h6></center><br>
#@markdown <center><code>Creat an account and Get Free Token  <a href="https://dashboard.ngrok.com/auth">Here</a></code>  
#@markdown <center><h6>Auto upload to gdrive after task completed. if gdrive is mounted<br>Default output path /content/downloads/</h6></center><br>
import os, urllib.request, pathlib
from IPython.display import clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/totalleecher/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    #updateCheck,
    runSh,
    findProcess,
    loadingAn,
    PortForward_wrapper
)
#if updateCheck("Checking updates ...", Version):  # VERSION CHECKING ...
#    !kill -9 -1 &
#clear_output()

loadingAn()

pathlib.Path('downloads').mkdir(exist_ok=True)
pathlib.Path(f"{HOME}/.config/deluge/").mkdir(parents=True, exist_ok=True)

if not (findProcess("/usr/bin/python", "deluged") or findProcess("/usr/bin/python", "deluge-web")):
  runSh('sudo apt install -y deluged deluge-console deluge-webui')
  runSh(
        f"wget -qq https://raw.githubusercontent.com/totalleecher/Google-Colab-CloudTorrent/master/res/deluge/core.conf \
            -O {HOME}/.config/deluge/core.conf"
    )
  runSh(
        f"wget -qq https://raw.githubusercontent.com/totalleecher/Google-Colab-CloudTorrent/master/res/deluge/web.conf \
            -O {HOME}/.config/deluge/web.conf"
    )
  runSh('deluged &> /dev/null &', shell=True)
  runSh('deluge-web --fork', shell=True)
  runSh("""sed -i 's/if s.hexdigest() == config\["pwd_sha1"\]:/if True:/' /usr/lib/python2.7/dist-packages/deluge/ui/web/auth.py""")
  runSh("sed -i 's/onShow:function(){this.passwordField.focus(.*)}/onShow:function(){this.onLogin();}/' /usr/lib/python2.7/dist-packages/deluge/ui/web/js/deluge-all.js")

# START_SERVER
clear_output()
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['deluge', 8112, 'http'],
                            ['utorrent', 5454, 'http']], 'eu', 
    [f"{HOME}/.ngrok2/delugeUtorrent.yml", 4042]
).start('deluge')


## <img src='https://avatars1.githubusercontent.com/u/3368377?s=400&v=4' height="30" alt="logo"/><img src='https://raw.githubusercontent.com/linuxserver/docker-templates/master/linuxserver.io/img/nzbget-banner.png' height="30" alt="nzbget"/>

In [ ]:
#@markdown <h3>← Click Here to Use NZBget (<i>ngrok</i>)</h3>
#@markdown <center>Creat an account and Get Free Token  <a href="https://dashboard.ngrok.com/auth">Here</a>  
Ngrok_Token = "" #@param {type:"string"}

import os, psutil, uuid, time, urllib.request, json; from IPython.display import clear_output
import ipywidgets as widgets
from IPython.display import HTML, clear_output
from google.colab import output

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

if os.path.isfile("/usr/local/bin/ngrok") == False:
  !wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip -qq -n ngrok-stable-linux-amd64.zip
  !mv ngrok /usr/local/bin/ngrok
  !rm -f /content/ngrok-stable-linux-amd64.zip
  print("ngrok successfully installed.")
  clear_output(wait=True)
else:
  print("ngrok already installed. Skipping... ")
  clear_output(wait=True)
  !pkill ngrok

def RandomGenerator():
  return time.strftime("%S") + str(time.time()).split(".")[-1]

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue

try:
  try:
    Random_NumberNZ
  except NameError:
    Random_NumberNZ = RandomGenerator()
    display(MakeLabel('Installing in Progress', 'warning'))
  if os.path.isfile("/content/nzbget/nzbget") == False:
    !mkdir /content/nzbget
    get_ipython().system_raw("rm -rf /content/sample_data/ && wget https://nzbget.net/download/nzbget-latest-bin-linux.run && sh nzbget-latest-bin-linux.run")
  if CheckProcess("nzbget", "") != True:
    get_ipython().system_raw("/content/nzbget/nzbget -s &")
except:
  clear_output(wait=True)

get_ipython().system_raw('/usr/local/bin/ngrok authtoken $Ngrok_Token && /usr/local/bin/ngrok http 6789 &')
time.sleep(10)
clear_output(wait=True)
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  host = data['tunnels'][0]['public_url'][8:]
  print(f'Open this link in new tab http://{host}')

## <img src='https://biplobsd.github.io/RLabClone/img/title_rclonelab.svg' height="45" alt="RcloneLab"/> <font size=-4>___RcloneLab___</font>



In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Execute rClone

Mode = "Copy"  # @param ["Move", "Copy", "Sync", "Verify", "Dedupe", "Clean Empty Dirs", "Empty Trash"]
Source = ""  # @param {type:"string"}
Destination = ""  # @param {type:"string"}
Extra_Arguments = ""  # @param {type:"string"}
COPY_SHARED_FILES = False  # @param{type: "boolean"}
Compare = "Size & Checksum"
TRANSFERS, CHECKERS = 20, 20
THROTTLE_TPS = True
BRIDGE_TRANSFER = False  # @param{type: "boolean"}
FAST_LIST = False  # @param{type: "boolean"}
OPTIMIZE_GDRIVE = True
SIMPLE_LOG = True
RECORD_LOGFILE = False  # @param{type: "boolean"}
SKIP_NEWER_FILE = False
SKIP_EXISTED = False
SKIP_UPDATE_MODTIME = False
ONE_FILE_SYSTEM = False
LOG_LEVEL = "DEBUG"
SYNC_MODE = "Delete after transfering"
SYNC_TRACK_RENAME = True
DEDUPE_MODE = "Largest"
USE_TRASH = True
DRY_RUN = False  # @param{type: "boolean"}
# ================================================================ #
from os import path as _p

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run

    shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))

from datetime import datetime as _dt
from rlab_utils import (
    displayOutput,
    checkAvailable,
    runSh,
    prepareSession,
    PATH_RClone_Config,
    accessSettingFile,
    memGiB,
)


def populateActionArg():
    if Mode == "Copy":
        actionArg = "copy"
    elif Mode == "Sync":
        actionArg = "sync"
    elif Mode == "Verify":
        actionArg = "check"
    elif Mode == "Dedupe":
        actionArg = "dedupe largest"
    elif Mode == "Clean Empty Dirs":
        actionArg = "rmdirs"
    elif Mode == "Empty Trash":
        actionArg = "delete"
    else:
        actionArg = "move"

    return actionArg


def populateCompareArg():
    if Compare == "Mod-Time":
        compareArg = "--ignore-size"
    elif Compare == "Size":
        compareArg = "--size-only"
    elif Compare == "Checksum":
        compareArg = "-c --ignore-size"
    else:
        compareArg = "-c"

    return compareArg


def populateOptimizeGDriveArg():
    return (
        "--buffer-size 256M \
            --drive-chunk-size 256M \
                --drive-upload-cutoff 256M \
                    --drive-acknowledge-abuse \
                        --drive-keep-revision-forever"
        if OPTIMIZE_GDRIVE
        else "--buffer-size 128M"
    )


def populateGDriveCopyArg():
    if BRIDGE_TRANSFER and memGiB() < 13:
        global TRANSFERS, CHECKERS
        TRANSFERS, CHECKERS = 10, 80
    else:
        pass
    return "--disable copy" if BRIDGE_TRANSFER else "--drive-server-side-across-configs"


def populateStatsArg():
    statsArg = "--stats-one-line --stats=5s" if SIMPLE_LOG else "--stats=5s -P"
    if LOG_LEVEL != "OFF":
        statsArg += " -v" if SIMPLE_LOG else "-vv"
    elif LOG_LEVEL == "INFO":
        statsArg += " --log-level INFO"
    elif LOG_LEVEL == "ERROR":
        statsArg += " --log-level ERROR"
    else:
        statsArg += " --log-level DEBUG"
    return statsArg


def populateSyncModeArg():
    if Mode != "Sync":
        return ""
    elif SYNC_MODE == "Delete before transfering":
        syncModeArg = "--delete-before"
    elif SYNC_MODE == "Delete after transfering":
        syncModeArg = "--delete-after"
    else:
        syncModeArg = "--delete-during"
    if SYNC_TRACK_RENAME:
        syncModeArg += " --track-renames"
    return syncModeArg


def populateDedupeModeArg():
    if DEDUPE_MODE == "Interactive":
        dedupeModeArg = "--dedupe-mode interactive"
    elif DEDUPE_MODE == "Skip":
        dedupeModeArg = "--dedupe-mode skip"
    elif DEDUPE_MODE == "First":
        dedupeModeArg = "--dedupe-mode first"
    elif DEDUPE_MODE == "Newest":
        dedupeModeArg = "--dedupe-mode newest"
    elif DEDUPE_MODE == "Oldest":
        dedupeModeArg = "--dedupe-mode oldest"
    elif DEDUPE_MODE == "Rename":
        dedupeModeArg = "--dedupe-mode rename"
    else:
        dedupeModeArg = "--dedupe-mode largest"

    return dedupeModeArg


def generateCmd():
    sharedFilesArgs = (
        "--drive-shared-with-me --files-from /content/upload.txt --no-traverse"
        if COPY_SHARED_FILES
        else ""
    )

    logFileArg = f"--log-file /content/rclone_log.txt -vv -P"

    args = [
        "rclone",
        f"--config {PATH_RClone_Config}/rclone.conf",
        '--user-agent "Mozilla"',
        populateActionArg(),
        f'"{Source}"',
        f'"{Destination}"' if Mode in ("Move", "Copy", "Sync") else "",
        f"--transfers {str(TRANSFERS)}",
        f"--checkers {str(CHECKERS)}",
    ]

    if Mode == "Verify":
        args.append("--one-way")
    elif Mode == "Empty Trash":
        args.append("--drive-trashed-only --drive-use-trash=false")
    else:
        args.extend(
            [
                populateGDriveCopyArg(),
                populateSyncModeArg(),
                populateCompareArg(),
                populateOptimizeGDriveArg(),
                "-u" if SKIP_NEWER_FILE else "",
                "--ignore-existing" if SKIP_EXISTED else "",
                "--no-update-modtime" if SKIP_UPDATE_MODTIME else "",
                "--one-file-system" if ONE_FILE_SYSTEM else "",
                "--tpslimit 95 --tpslimit-burst 40" if THROTTLE_TPS else "",
                "--fast-list" if FAST_LIST else "",
                "--delete-empty-src-dirs" if Mode == "Move" else "",
            ]
        )
    args.extend(
        [
            "-n" if DRY_RUN else "",
            populateStatsArg() if not RECORD_LOGFILE else logFileArg,
            sharedFilesArgs,
            Extra_Arguments,
        ]
    )

    return args


def executeRclone():
    prepareSession()
    if Source.strip() == "":
        displayOutput("❌ The Source field is empty.")
        return
    if checkAvailable("/content/rclone_log.txt"):
        if not checkAvailable("/content/logfiles"):
            runSh("mkdir -p -m 666 /content/logfiles")
        job = accessSettingFile("job.txt")
        runSh(
            f'mv /content/rclone_log.txt /content/logfiles/{job["title"]}_{job["status"]}_logfile.txt'
        )

    onGoingJob = {
        "title": f'{Mode}_{Source}_{Destination}_{_dt.now().strftime("%a-%H-%M-%S")}',
        "status": "ongoing",
    }
    accessSettingFile("job.txt", onGoingJob)

    cmd = " ".join(generateCmd())
    runSh(cmd, output=True)
    displayOutput(Mode, "success")

    onGoingJob["status"] = "finished"
    accessSettingFile("job.txt", onGoingJob)

executeRclone()


In [ ]:
#@markdown <center><h3>Rclone MOUNT / UNMOUNT</h3>Mount the remote as file system on a mountpoint.</center>
Cache_Directory = "DISK" #@param ["RAM", "DISK"]
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
#####################################
if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from rlab_utils import (
    runSh,
    prepareSession,
    PATH_RClone_Config,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))

prepareSession()
content = open(f"{PATH_RClone_Config}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {PATH_RClone_Config}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --buffer-size 256M' \
      ' --transfers 10' \
      ' --vfs-cache-mode full' \
      ' --vfs-cache-max-age 0h0m1s' \
      ' --vfs-cache-poll-interval 0m1s' \
      f' --cache-dir {cache_path}' \
      ' --allow-other' \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

In [ ]:
#@markdown <center><h3>Rclone webui</h3></center><br>
import os, signal
import random
import string
import urllib.request
from IPython.display import HTML, clear_output
from urllib.parse import urlparse
import time

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
#@markdown Default <br>&emsp;username : user<br>&emsp;password : pass
HOME = os.path.expanduser("~")
runW = get_ipython()

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/totalleecher/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    displayUrl,
    findProcess,
    CWD,
    textAn,
    checkAvailable
)
from rlab_utils import (
    displayOutput,
    checkAvailable,
    prepareSession,
    PATH_RClone_Config,
    accessSettingFile,
    memGiB,
)

loadingAn()
prepareSession()
pid = findProcess("rclone", "rcd", isPid=True)
try:
  os.kill(int(pid), signal.SIGTERM)
except TypeError:
  pass
cmd = "rclone rcd --rc-web-gui --rc-addr :5572" \
      " --rc-serve" \
      " --rc-user=user --rc-pass=pass" \
      " --rc-no-auth" \
      rf" --config {PATH_RClone_Config}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --transfers 16' \
      " &"
runSh(cmd, shell=True)
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rclone', 5572, 'http'],
                            ['filebrowser', 4000, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/filebrowserRclone.yml", 4099]
).start('rclone', displayB=False)
# output
clear_output()
displayUrl(Server, pNamU='Rclone webUI : ', 
           ExUrl=fr"https://user:pass@{urlparse(Server['url']).hostname}")

In [ ]:
#@markdown <center><h3>Create/Edit Rclone config</h3>Create a new remote with name, type and options.<br><font size=1px>After created your config file download that. Next time just upload and you are done!</font></center>
import os, urllib.request
from IPython.display import HTML
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "SA" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
HOME = os.path.expanduser("~")
runW = get_ipython()

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/totalleecher/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

#####################################

if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from rlab_utils import (
    prepareSession,
    PATH_RClone_Config,
    runSh
)
from ocr import (
    PortForward_wrapper
)

###################################
import codecs
import contextlib
import locale
import os
import pty
import select
import signal
import subprocess
import sys
import termios
import time


from IPython.utils import text
import six
from google.colab import _ipython
from google.colab import _message
from google.colab.output import _tags

# Linux read(2) limits to 0x7ffff000 so stay under that for clarity.
_PTY_READ_MAX_BYTES_FOR_TEST = 2**20  # 1MB

_ENCODING = 'UTF-8'

class ShellResult(object):
  """Result of an invocation of the shell magic.

  Note: This is intended to mimic subprocess.CompletedProcess, but has slightly
  different characteristics, including:
    * CompletedProcess has separate stdout/stderr properties. A ShellResult
      has a single property containing the merged stdout/stderr stream,
      providing compatibility with the existing "!" shell magic (which this is
      intended to provide an alternative to).
    * A custom __repr__ method that returns output. When the magic is invoked as
      the only statement in the cell, Python prints the string representation by
      default. The existing "!" shell magic also returns output.
  """

  def __init__(self, args, returncode, command_output):
    self.args = args
    self.returncode = returncode
    self.output = command_output

  def check_returncode(self):
    if self.returncode:
      raise subprocess.CalledProcessError(
          returncode=self.returncode, cmd=self.args, output=self.output)

  def _repr_pretty_(self, p, cycle):  # pylint:disable=unused-argument
    # Note: When invoking the magic and not assigning the result
    # (e.g. %shell echo "foo"), Python's default semantics will be used and
    # print the string representation of the object. By default, this will
    # display the __repr__ of ShellResult. Suppress this representation since
    # the output of the command has already been displayed to the output window.
    if cycle:
      raise NotImplementedError


def _configure_term_settings(pty_fd):
  term_settings = termios.tcgetattr(pty_fd)
  # ONLCR transforms NL to CR-NL, which is undesirable. Ensure this is disabled.
  # http://man7.org/linux/man-pages/man3/termios.3.html
  term_settings[1] &= ~termios.ONLCR

  # ECHOCTL echoes control characters, which is undesirable.
  term_settings[3] &= ~termios.ECHOCTL

  termios.tcsetattr(pty_fd, termios.TCSANOW, term_settings)


def _run_command(cmd, clear_streamed_output):
  """Calls the shell command, forwarding input received on the stdin_socket."""
  locale_encoding = locale.getpreferredencoding()
  if locale_encoding != _ENCODING:
    raise NotImplementedError(
        'A UTF-8 locale is required. Got {}'.format(locale_encoding))

  parent_pty, child_pty = pty.openpty()
  _configure_term_settings(child_pty)

  epoll = select.epoll()
  epoll.register(
      parent_pty,
      (select.EPOLLIN | select.EPOLLOUT | select.EPOLLHUP | select.EPOLLERR))

  try:
    temporary_clearer = _tags.temporary if clear_streamed_output else _no_op

    with temporary_clearer(), _display_stdin_widget(
        delay_millis=500) as update_stdin_widget:
      # TODO(b/115531839): Ensure that subprocesses are terminated upon
      # interrupt.
      p = subprocess.Popen(
          cmd,
          shell=True,
          executable='/bin/bash',
          stdout=child_pty,
          stdin=child_pty,
          stderr=child_pty,
          close_fds=True)
      # The child PTY is only needed by the spawned process.
      os.close(child_pty)

      return _monitor_process(parent_pty, epoll, p, cmd, update_stdin_widget)
  finally:
    epoll.close()
    os.close(parent_pty)


class _MonitorProcessState(object):

  def __init__(self):
    self.process_output = six.StringIO()
    self.is_pty_still_connected = True


def _monitor_process(parent_pty, epoll, p, cmd, update_stdin_widget):
  """Monitors the given subprocess until it terminates."""
  state = _MonitorProcessState()

  # A single UTF-8 character can span multiple bytes. os.read returns bytes and
  # could return a partial byte sequence for a UTF-8 character. Using an
  # incremental decoder is incrementally fed input bytes and emits UTF-8
  # characters.
  decoder = codecs.getincrementaldecoder(_ENCODING)()

  num_interrupts = 0
  echo_status = None
  while True:
    try:
      result = _poll_process(parent_pty, epoll, p, cmd, decoder, state)
      if result is not None:
        return result
      term_settings = termios.tcgetattr(parent_pty)
      new_echo_status = bool(term_settings[3] & termios.ECHO)
      if echo_status != new_echo_status:
        update_stdin_widget(new_echo_status)
        echo_status = new_echo_status
    except KeyboardInterrupt:
      try:
        num_interrupts += 1
        if num_interrupts == 1:
          p.send_signal(signal.SIGINT)
        elif num_interrupts == 2:
          # Process isn't responding to SIGINT and user requested another
          # interrupt. Attempt to send SIGTERM followed by a SIGKILL if the
          # process doesn't respond.
          p.send_signal(signal.SIGTERM)
          time.sleep(0.5)
          if p.poll() is None:
            p.send_signal(signal.SIGKILL)
      except KeyboardInterrupt:
        # Any interrupts that occur during shutdown should not propagate.
        pass

      if num_interrupts > 2:
        # In practice, this shouldn't be possible since
        # SIGKILL is quite effective.
        raise


def _poll_process(parent_pty, epoll, p, cmd, decoder, state):
  """Polls the process and captures / forwards input and output."""

  terminated = p.poll() is not None
  if terminated:
    termios.tcdrain(parent_pty)
    # We're no longer interested in write events and only want to consume any
    # remaining output from the terminated process. Continuing to watch write
    # events may cause early termination of the loop if no output was
    # available but the pty was ready for writing.
    epoll.modify(parent_pty,
                 (select.EPOLLIN | select.EPOLLHUP | select.EPOLLERR))

  output_available = False

  events = epoll.poll()
  input_events = []
  for _, event in events:
    if event & select.EPOLLIN:
      output_available = True
      raw_contents = os.read(parent_pty, _PTY_READ_MAX_BYTES_FOR_TEST)
      import re
      decoded_contents = re.sub(r"http:\/\/127.0.0.1:53682", Server["url"], 
                                decoder.decode(raw_contents))
      sys.stdout.write(decoded_contents)
      state.process_output.write(decoded_contents)

    if event & select.EPOLLOUT:
      # Queue polling for inputs behind processing output events.
      input_events.append(event)

    # PTY was disconnected or encountered a connection error. In either case,
    # no new output should be made available.
    if (event & select.EPOLLHUP) or (event & select.EPOLLERR):
      state.is_pty_still_connected = False

  for event in input_events:
    # Check to see if there is any input on the stdin socket.
    # pylint: disable=protected-access
    input_line = _message._read_stdin_message()
    # pylint: enable=protected-access
    if input_line is not None:
      # If a very large input or sequence of inputs is available, it's
      # possible that the PTY buffer could be filled and this write call
      # would block. To work around this, non-blocking writes and keeping
      # a list of to-be-written inputs could be used. Empirically, the
      # buffer limit is ~12K, which shouldn't be a problem in most
      # scenarios. As such, optimizing for simplicity.
      input_bytes = bytes(input_line.encode(_ENCODING))
      os.write(parent_pty, input_bytes)

  # Once the process is terminated, there still may be output to be read from
  # the PTY. Wait until the PTY has been disconnected and no more data is
  # available for read. Simply waiting for disconnect may be insufficient if
  # there is more data made available on the PTY than we consume in a single
  # read call.
  if terminated and not state.is_pty_still_connected and not output_available:
    sys.stdout.flush()
    command_output = state.process_output.getvalue()
    return ShellResult(cmd, p.returncode, command_output)

  if not output_available:
    # The PTY is almost continuously available for reading input to provide
    # to the underlying subprocess. This means that the polling loop could
    # effectively become a tight loop and use a large amount of CPU. Add a
    # slight delay to give resources back to the system while monitoring the
    # process.
    # Skip this delay if we read output in the previous loop so that a partial
    # read doesn't unnecessarily sleep before reading more output.
    # TODO(b/115527726): Rather than sleep, poll for incoming messages from
    # the frontend in the same poll as for the output.
    time.sleep(0.1)


@contextlib.contextmanager
def _display_stdin_widget(delay_millis=0):
  """Context manager that displays a stdin UI widget and hides it upon exit.

  Args:
    delay_millis: Duration (in milliseconds) to delay showing the widget within
      the UI.

  Yields:
    A callback that can be invoked with a single argument indicating whether
    echo is enabled.
  """
  shell = _ipython.get_ipython()
  display_args = ['cell_display_stdin', {'delayMillis': delay_millis}]
  _message.blocking_request(*display_args, parent=shell.parent_header)

  def echo_updater(new_echo_status):
    # Note: Updating the echo status uses colab_request / colab_reply on the
    # stdin socket. Input provided by the user also sends messages on this
    # socket. If user input is provided while the blocking_request call is still
    # waiting for a colab_reply, the input will be dropped per
    # https://github.com/googlecolab/colabtools/blob/56e4dbec7c4fa09fad51b60feb5c786c69d688c6/google/colab/_message.py#L100.
    update_args = ['cell_update_stdin', {'echo': new_echo_status}]
    _message.blocking_request(*update_args, parent=shell.parent_header)

  yield echo_updater

  hide_args = ['cell_remove_stdin', {}]
  _message.blocking_request(*hide_args, parent=shell.parent_header)


@contextlib.contextmanager
def _no_op():
  yield

###################################
prepareSession()

PORT_FORWARD = "localhost" #@param ["ngrok", "localhost"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rcloneConfig', 53682, 'http'], 
                            ['pyload', 8000, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/rclonePyload.yml", 4074]
).start('rcloneConfig', displayB=False, v=False)

printData = """Copy this URL,
  It's needed for authentication purposes.
  After completing your account select, you redirect to a website, after back
  you need to change http://127.0.0.0:53682 to {}""".format(Server['url'])
print(printData)
display(HTML('&emsp;&emsp;<a href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/rclone_config_create.gif" target="_blank">See how</a><br><br>'))
print(f"{Server['url']}", end="\n\n")
_run_command(f"rclone config --config {PATH_RClone_Config}/rclone.conf", False)

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Execute Upload Local File
MODE = "RCONFIG"  # @param ['UTILS', 'RCONFIG', 'RCONFIG_append', "GENERATELIST"]
REMOTE = "mnc"  # @param {type:"string"}
QUERY_PATTERN = ""  # @param {type:"string"}
# @markdown #### <font size=1px>For not able to upload local file : https://stackoverflow.com/a/58661947</font>
# ================================================================ #

from os import path as _p

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))  # nosec

import importlib, rlab_utils
from google.colab import files  # pylint: disable=import-error #nosec
from rlab_utils import checkAvailable, runSh, PATH_RClone_Config, prepareSession


def generateUploadList():
    prepareSession()
    if checkAvailable("/content/upload.txt"):
        runSh("rm -f upload.txt")
    runSh(
        f"rclone --config {PATH_RClone_Config}/rclone.conf lsf {REMOTE}: --include '{QUERY_PATTERN}' --drive-shared-with-me --files-only --max-depth 1 > /content/upload.txt",
        shell=True,  # nosec
    )


def uploadLocalFiles():
    prepareSession()
    if MODE == "UTILS":
        filePath = "/root/.ipython/rlab_utils.py"
    elif MODE in ("RCONFIG", "RCONFIG_append"):
        filePath = f"{PATH_RClone_Config}/rclone.conf"
    else:
        pass

    try:
      if checkAvailable(filePath):
        runSh(f"rm -f {filePath}")
      print("Select file from your computer.\n")
      uploadedFile = files.upload()
      fileNameDictKeys = uploadedFile.keys()
      fileNo = len(fileNameDictKeys)
      if fileNo > 1:
          for fn in fileNameDictKeys:
              runSh(f'rm -f "/content/{fn}"')
          return print("\nPlease only upload a single config file.")
      elif fileNo == 0:
          return print("\nFile upload cancelled.")
      elif fileNo == 1:
          for fn in fileNameDictKeys:
              if checkAvailable(f"/content/{fn}"):
                  if MODE == "RCONFIG_append":
                    import urllib
                    urllib.request.urlretrieve("https://biplobsd.github.io/RLabClone/res/rclonelab/rclone.conf",
                                               "/usr/local/sessionSettings/rclone.conf")
                    with open(f"/content/{fn}", 'r+') as r:
                      new_data = r.read()
                      runSh(f'rm -f "/content/{fn}"')
                    with open(filePath, 'r+') as f:
                      old_data = f.read()
                      f.seek(0)
                      f.truncate(0)
                      f.write(old_data + new_data)
                    print("\nUpdate completed.")
                  else:
                    runSh(f'mv -f "/content/{fn}" {filePath}')
                    runSh(f"chmod 666 {filePath}")
                    runSh(f'rm -f "/content/{fn}"')
                    importlib.reload(rlab_utils)
                    print("\nUpload completed.")
              return
      else:
          print("\nNo file")
          return
    except:
      return print("\nUpload process Error.")


if MODE == "GENERATELIST":
    generateUploadList()
else:
    uploadLocalFiles()


In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Download config file.
MODE = "RCONFIG"  # @param ['UTILS', 'RCONFIG']
# ================================================================ #
from google.colab import files

def downloadFile():
  if MODE == "UTILS":
      filePath = "/root/.ipython/rlab_utils.py"
  elif MODE == "RCONFIG":
      filePath = f"{PATH_RClone_Config}/rclone.conf"
  else:
      pass
  try:
    files.download(filePath)
  except FileNotFoundError:
    print("File not found!")

if __name__ == "__main__":
  downloadFile()


# <center><b>Special Features</b><h5>Zipping, Encoding, Cloning</center></h5>

## <img src='https://upload.wikimedia.org/wikipedia/commons/5/5f/FFmpeg_Logo_new.svg' height="30" alt="FFMPEG"/>

In [ ]:
#@title <font size="5"><i>Upgrade FFmpeg to Latest Version</font> { vertical-output: true }
#@markdown <h3>← Click Here to Install FFmpeg, VCSI, Mkvtoolnix
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/totalleecher/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')

os.system('sudo add-apt-repository ppa:savoury1/ffmpeg4')
os.system('sudo apt-get update')
os.system('sudo apt-get install ffmpeg')
os.system('apt-get install mkvtoolnix')
os.system('pip install vcsi')


clear_output()
print("Install Finished")

In [ ]:
#@markdown <h3>← Click Here to Check ffmpeg Version</h3>
!ffmpeg 



In [ ]:
#@markdown <h3>← Media Info</h3>

path = "" #@param {type:"string"}
save_txt = False #@param {type:"boolean"}
import os, uuid, re, IPython
import ipywidgets as widgets
import time

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output, drive

def mediainfo():
  display(HTML("<br>"))
#   print(path.split("/")[::-1][0])
  display(HTML("<br>"))
#   media = !mediainfo "$path"
#   media = "\n".join(media).replace(os.path.dirname(path)+"/", "")
  get_ipython().system_raw("""mediainfo --LogFile="/root/.nfo" "$path" """)
  with open('/root/.nfo', 'r') as file:
    media = file.read()
    media = media.replace(os.path.dirname(path)+"/", "")
  print(media)
  get_ipython().system_raw("rm -f '/root/.nfo'")
  
  if save_txt:
    txt = path.rpartition('.')[0] + ".txt"
    if os.path.exists(txt):
      get_ipython().system_raw("rm -f '$txt'")
    !curl -s https://pastebin.com/raw/TApKLQfM -o "$txt"
    with open(txt, 'a+') as file:
      file.write("\n\n")
      file.write(media)

while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output()
    
if not os.path.exists("/usr/bin/mediainfo"):
  get_ipython().system_raw("apt-get install mediainfo")
  
mediainfo()

In [ ]:
import os, sys, re
#@markdown <b>Encoder</b>
Encoder = "CPU" #@param ["GPU", "CPU"]
codec = "x264" #@param ["x264", "x265"]
#@markdown <b>Encoding all videos in folder</b>
video_folder_path = '' #@param {type:"string"}
#@markdown ---
#@markdown <b>Encoding selected videos</b>
video_file_path1 = "/content/drive/Shareddrives/(01Heroku)/Heroku/Be My Slave (Director's Cut).2012.BDRip.576p.x264.AAC.mp4-muxed.mp4" #@param {type:"string"}
video_file_path2 = '' #@param {type:"string"}
video_file_path3 = '' #@param {type:"string"}
video_file_path4 = '' #@param {type:"string"}
video_file_path5 = '' #@param {type:"string"}

#counting
if video_file_path1 != "":
  coa = 1
else:
  coa = 0

if video_file_path2 != "":
  cob = 1
else:
  cob = 0

if video_file_path3 != "":
  coc = 1
else:
  coc = 0

if video_file_path4 != "":
  cod = 1
else:
  cod = 0

if video_file_path5 != "":
  coe = 1
else:
  coe = 0

#@markdown ---
resolution = '360p' #@param ["2160p","1440p","1080p", "720p", "480p", "360p", "240p", "same as input","custom"]
#@markdown custom resloution (width)
custom_resolution = '480' #@param {type:"string"}
#@markdown ---
encode_setting = 'Advance' #@param ["Advance", "HEVC", "HEVC 10 Bit"]
file_type = 'mkv' #@param ["mkv", "mp4"]
rip_audio = False #@param {type:"boolean"}
rip_subtitle = False #@param {type:"boolean"}

if rip_audio == False:
  rip_audio_string =  "-acodec copy"
else:
  rip_audio_string =  "-an"

if rip_subtitle == False:
  rip_subtitle_string = "-scodec copy"
else:
  rip_subtitle_string = "-sn"


if resolution == '2160p':
   w = '3840'
elif resolution == '1440p':
   w = '2560'
elif resolution == '1080p':
   w = '1980'
elif resolution == '720p':
   w = '1280'
elif resolution == '480p':
   w = '854'
elif resolution == '360p':
   w = '640'
elif resolution == '240p':
   w = '426'
elif resolution == 'custom':
   w = custom_resolution
else:
   w = ''

if w != '':
#if  (w == '3840' or w == '2560' or w == '1980' or w == '1280' or w == '854' or w == '640' or w == '426'):
  scale_string = "-vf  scale="+(w)+":-1:flags=lanczos" 
else:
  scale_string = ""

ext = ".mp4",".MP4",".MTS",".mts",".m2ts",".mkv",".avi",".MOV",".mov",".wmv",".WMV",".flv",".mpg",".webm",".WEBM"
# As file at filePath is deleted now, so we should check if file exists or not not before deleting them
filePath = "ffmpeg.txt"
if os.path.exists(filePath):
     os.remove(filePath)

if video_folder_path == "":
 #try:
  f = open("ffmpeg.txt", "+w")
  x = (video_file_path1) + "\n" + (video_file_path2) + "\n" +(video_file_path3) + "\n" +(video_file_path4) +"\n" + (video_file_path5)
  f.write(x)
  f.close()
  count = coa+cob+coc+cod+coe
 #except:
  #err = 1

else:
#writing temp file
 for file in os.listdir(video_folder_path):
    if file.endswith(tuple(ext)):
        
     x = os.path.join(video_folder_path, file) 
     #print(x)
     print(x, file=open("ffmpeg.txt", "+a"))    

#counting line
 thefilepath = "ffmpeg.txt"
 count = len(open(thefilepath).readlines(  ))
#@markdown ---
#@markdown <code>Setting only for Tesla T4 or Turing GPU</code> <i>(Please uncheck for P100 or K80 GPU)</i>
My_GPU_is_Tesla_T4 = False #@param {type:"boolean"}
bframes = 3 #@param {type:"slider", min:0, max:4, step:1}
#@markdown <b>Tip : </b><i>(tune = 0: High quality, 1: Low Latency, 2: Ultra Low Latency, 3: Lossless)</i>
tune = 0 #@param {type:"slider", min:0, max:4, step:1}
if My_GPU_is_Tesla_T4 == True:
  bframe_string = "-bf:v "+str(bframes)+" -b_ref_mode:v middle"
  tune_string = "-tune "+str(tune)
else:
  bframe_string =""
  tune_string =""
#@markdown ---
#@markdown <h4>Advance Settings</h4>
#@markdown <b>Video Setting</b>
preset = 'slow' #@param ["slow", "medium", "fast", "hq", "hp", "bd", "ll", "llhq", "llhp", "lossless", "losslesshp"]
level = '5.2' #@param ["default","4.1", "5.1", "5.2", "6.2"]
tier = 'main' #@param ["default","main", "high"]
#@markdown <code>Setting only for GPU Encoding</code>
profile = 'main' #@param ["main", "main10", "rext"]
pixfmt = 'p010le' #@param ["nv12", "yuv420p", "p010le", "yuv444p", "p016le", "yuv444p16le"]
rc = 'vbr_hq' #@param ["vbr", "cbr", "vbr_2pass", "ll_2pass_size", "vbr_hq", "cbr_hq"]
rcla = '32' #@param ["8", "16", "32", "64"]
overall_bitrate = 2500 #@param {type:"slider", min:500, max:10000, step:100}
max_bitrate = 20000 #@param {type:"slider", min:500, max:50000, step:100}
buffer_size = 60000 #@param {type:"slider", min:500, max:90000, step:100}
deblock = -3 #@param {type:"slider", min:-6, max:6, step:1}
reframe = 5 #@param {type:"slider", min:1, max:6, step:1}
surfaces = 64 #@param {type:"slider", min:0, max:64, step:1}

#@markdown <code>Setting only for CPU Encoding</code>
profile_cpu = 'main10' #@param ["main10"]
pixfmt_cpu = 'yuv420p10le' #@param ["yuv420p","yuv420p10le","yuv444p","yuv444p16le"]
threads = 40 #@param {type:"slider", min:0, max:40, step:1}
crf = 28 #@param {type:"slider", min:0, max:30, step:1}





if level != "default":
  l_string =  "-level "+str(level)
else:
  l_string =""

if tier != "default":
 t_string =  "-tier "+str(tier)
else:
 t_string = ""

#tp = '1' #@param ["0", "1"]
#cq = '21' #@param {type:"string"}
#qm ='21' #@param {type:"string"}
#qmx = '27' #@param {type:"string"}
#qp = '23' #@param {type:"string"}
#qb = '25' #@param {type:"string"}
#qi = '21' #@param {type:"string"}

#@markdown <b>Audio Setting</b>

audio_output = 'No audio' #@param ["None", "copy", "flac", "aac", "libopus", "eac3", "No audio", "same as input"]
channel = 'same as input' #@param ["DownMix 2CH", "same as input"]

if audio_output == "same as input":
  audio_string = "-acodec copy"
elif audio_output == "No audio":
  audio_string = "-an"
elif audio_output == "None":
  audio_string = ""
else:
  audio_string = "-c:a "+(audio_output)

if channel == "DownMix 2CH":
  channel_string ="-ac 2"
else:
  channel_string =""

#@markdown <b>Subtitle Setting</b>
#@markdown <h5>Please use <code>ass</code> file for hardsub</h5>
hardsub = False #@param {type:"boolean"}
subtitle_option = 'same as input' #@param ["None","No sub", "Add custom sub","same as input"]
custom_subtitle_path = "" #@param {type:"string"}

#@markdown <b>Custom Added Setting</b>
custom_command = "" #@param {type:"string"}


if hardsub == False:

 if subtitle_option == "No sub":
  subtitle_string = "-sn"
 elif subtitle_option == "same as input":
  subtitle_string = "-scodec copy"
 elif subtitle_option == "None":
  subtitle_string = ""
 else:
  subtitle_string = "-i "+(custom_subtitle_path)

else:
  subtitle_string = "ass="+(custom_subtitle_path)
#=================
if custom_command != "":
 c_string = custom_command
else:
 c_string = ""
#=================

os.environ['ps'] = preset
os.environ['pf'] = profile
os.environ['pf_cpu'] = profile_cpu
os.environ['pfm'] = pixfmt
os.environ['pfmcpu'] = pixfmt_cpu
os.environ['br'] = str(overall_bitrate)
os.environ['max'] = str(max_bitrate)
os.environ['buff'] = str(buffer_size)
os.environ['de'] = str(deblock)
os.environ['ref'] = str(reframe)
os.environ['sur'] = str(surfaces)
os.environ['lv'] = l_string
os.environ['ti'] = t_string
os.environ['rc'] = rc
os.environ['rl'] = rcla
os.environ['thr'] = str(threads)
os.environ['crf'] = str(crf)
os.environ['res'] = resolution
#os.environ['tp'] = tp
#os.environ['cq'] = cq
#os.environ['qP'] = qp
#os.environ['qB'] = qb
#os.environ['qI'] = qi
#os.environ['qm'] = qm
#os.environ['qmx'] = qmx
os.environ['bfs'] = str(bframe_string)
os.environ['tus'] = str(tune_string)
os.environ['scs'] = str(scale_string)
os.environ['aus'] = audio_string
os.environ['chc'] = channel_string
os.environ['sus'] = subtitle_string
os.environ['cus'] = str(c_string)
#=================
#Batch Encoding
if count != 0:
 f=open('ffmpeg.txt')
 lines=f.readlines()

 i = 0
 while i < count:
  video_file_path = lines[i]
  video_file_path = video_file_path.rstrip("\n")
  #print(video_file_path)

  delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
  testsplit = video_file_path.split("/")
  filename = re.sub("^[\/]", "", delsplit.group(0))
  filename_raw = re.sub(".{4}$", "", filename)
  resolution_raw = re.search("[^p]{3,4}", resolution)
  output_file_path = re.search("^[\/].+\/", video_file_path)

  os.environ['inputFile'] = video_file_path
  os.environ['outputPath'] = output_file_path.group(0)
  os.environ['fileName'] = filename_raw
  os.environ['fileType'] = file_type
  os.environ['resolutionWidth'] = resolution_raw.group(0)

  if Encoder == "GPU":
   if codec == "x265":
    if encode_setting == "Advance":

     !ffmpeg  -hwaccel_output_format cuda -i "$inputFile" -stats -flags +loop -c:v hevc_nvenc -preset:v "$ps" -rc "$rc" -2pass 1 -b:v "$br"k -maxrate "$max"k -bufsize "$buff"k -profile:v "$pf" $lv $ti -pix_fmt "$pfm" -rc-lookahead "$rl" -no-scenecut 1 -temporal-aq 1 -deblock:v "$de":"$de" -refs:v "$ref" -surfaces "$sur" $bfs $tus $scs $aus $chs $sus $cus  "$outputPath"/"$fileName"-"$res"."$fileType" 
  
    elif encode_setting == "HEVC":
     !ffmpeg  -hwaccel_output_format cuda -i "$inputFile" -stats -flags +loop -c:v hevc_nvenc -preset:v slow -rc vbr_hq -2pass 1 -b:v 2500k -maxrate 20M -bufsize 60M -cq 1 -forced-idr 1 -nonref_p 1 -pix_fmt p010le -rc-lookahead 32 -no-scenecut 1 -temporal-aq 1 -deblock:v -3:-3 -refs:v 5  -surfaces 64 $bfs $tus $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 

    else:
     !ffmpeg  -hwaccel_output_format cuda -i "$inputFile" -stats -flags +loop -c:v hevc_nvenc -preset:v slow -rc vbr_hq -2pass 1 -b:v 2500k -maxrate 20M -bufsize 60M -cq 1 -forced-idr 1 -nonref_p 1 -profile:v main10 -pix_fmt p010le -rc-lookahead 32 -no-scenecut 1 -temporal-aq 1 -deblock:v -3:-3 -refs:v 5  -surfaces 64 $bfs $tus $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 
   else:
     !ffmpeg  -hwaccel_output_format cuda -i "$inputFile" -c:v h264_nvenc $bfs $tus $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 
     
  else:
   if codec == "x265":
    if encode_setting == "Advance":
     !ffmpeg -i "$inputFile" -flags +loop -c:v libx265 -profile:v "$pf_cpu" $lv $ti -pix_fmt "$pfmcpu" -threads "$thr" -thread_type frame -preset:v "$ps" -crf "$crf" -x265-params "rc-lookahead=40:bframes=4:b-adapt=2:ref=6:aq-mode=0:aq-strength=0:aq-motion=0:me=hex:subme=3:max-merge=3:weightb=1:no-fast-intra=1:tskip-fast=0:rskip=0:strong-intra-smoothing=0:b-intra=1:early-skip=0:sao=0:rd=1:psy-rd=0:deblock=-5,-5" $scs $aus $chs  $sus $cus "$outputPath"/"$fileName"-"$res"."$fileType" 
   
    elif encode_setting == "HEVC":
     !ffmpeg -i "$inputFile" -c:v libx265 -crf 28 -threads 6 -thread_type frame $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 

    else:
     !ffmpeg -i "$inputFile" -c:v libx265 -profile:v main10 -crf 28 -threads 6 -thread_type frame $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 
   else:
     !ffmpeg -hide_banner -i "$inputFile" -c:v libx264 -preset "$ps" -crf "$crf" -threads "$thr" -strict experimental $scs $aus $sus "$outputPath"/"$fileName"-"$res"."$fileType" 

  i += 1

 else:
  print("All Finished")
  os.remove(filePath)
else:
  print("Please input file or folder path")
#End of Code V3.0.0 - Codemater -  

###» Generate Thumbnails - Preview from Video 

In [ ]:
#@markdown <h3>← Click Here to generate thumbnail for all video in input folder path</h3>

import os
folder_path = "" #@param {type:"string"}
ext = ".mp4",".MP4",".MTS",".mts",".m2ts",".mkv",".avi",".MOV",".mov",".wmv",".WMV",".flv",".mpg",".webm",".WEBM"
video_path = '' #@param {type:"string"}


#counting
if video_path != "":
  count = 1
else:
  count = 0

# As file at filePath is deleted now, so we should check if file exists or not not before deleting them
filePath = "vcsi.txt"
if os.path.exists(filePath):
     os.remove(filePath)



if (folder_path == "") and (video_path != ""):
 #try:
  f = open("vcsi.txt", "+w")
  f.write(video_path)
  f.close()
  count = 1

elif (folder_path == "") and (video_path == ""):
  count = 0

else:
#writing temp file
 for file in os.listdir(folder_path):
    if file.endswith(tuple(ext)):
        
     x = os.path.join(folder_path, file) 
     #print(x)
     print(x, file=open("vcsi.txt", "+a"))    

#counting line
 thefilepath = "vcsi.txt"
 count = len(open(thefilepath).readlines(  ))


import os, sys, re
from IPython.display import Image, display
os.makedirs("/content/drive/My Drive/Thumbnail", exist_ok=True)

output_file_type = 'png' #@param ["png", "jpg"]
creation_engine = 'vcsi' #@param ["ffmpeg", "vcsi"]
output_path = 'same folder' #@param ["same folder", "My Drive/Thumbnail"]
#@markdown <code>Eg : gird 3 = 3x3</code>
grid = 4 #@param {type:"slider", min:1, max:20, step:1}
default_grid = True #@param {type:"boolean"}
time_stamp = False #@param {type:"boolean"}


if time_stamp == True:
  t_string = "-t"
else:
  t_string = ""

if default_grid == False:
  g_string = "-g " + str(grid) + "x" + str(grid) 
else:
  g_string = ""

os.environ['ts'] = t_string
os.environ['gs'] = g_string
#Batch Encoding
if count != 0:
 f=open('vcsi.txt')
 lines=f.readlines()

 i = 0
 while i < count:
  video_file_path = lines[i]
  video_file_path = video_file_path.rstrip("\n")
  print(video_file_path)
  
  output_file_path = re.search("^[\/].+\/", video_file_path)
  output_file_path_raw = output_file_path.group(0)
  delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
  filename = re.sub("^[\/]", "", delsplit.group(0))
  filename_raw = re.sub(".{4}$", "", filename)
  file_extension = re.search(".{3}$", filename)
  file_extension_raw = file_extension.group(0)

  os.environ['inputFile'] = video_file_path
  os.environ['outputPath'] = output_file_path_raw
  os.environ['outputExtension'] = output_file_type
  os.environ['fileName'] = filename_raw
  os.environ['fileExtension'] = file_extension_raw

  if output_path == "same folder":
     if creation_engine == 'ffmpeg':
      !ffmpeg -hide_banner -i "$inputFile" -vframes 1 -q:v 0 -vf "select=not(mod(n\,200)),scale=-1:480,tile=3x2" -an "$outputPath"/"$fileName"_thumbnails."$outputExtension"

  if output_path == "same folder":
     if creation_engine == 'vcsi':
      !vcsi $ts $gs "$inputFile" -o "$outputPath"/"$fileName"_thumbnails."$outputExtension"

  if not output_path == "same folder":
      !vcsi $ts $gs "$inputFile" -o "/content/drive/My Drive/Thumbnail"/"$fileName"_thumbnails."$outputExtension"

  i += 1

 else:
  print("All Finished")
  os.remove(filePath)
else:
  print("Please video file or folder path")
#End of Code V1.2 - Codemater -  

###» Misc.

In [ ]:
#@title Convert *.mkv* ➔ *.mp4* (Lossless)
import os, sys, re

video_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)

os.environ['inputFile'] = video_file_path
os.environ['outputFile'] = filename_raw
os.environ['outputPath'] = output_file_path_raw

!ffmpeg -hide_banner -i "$inputFile" -c copy -strict experimental "$outputPath""$outputFile".mp4

In [ ]:
#@title Convert Trim Video File (Lossless)

import os, sys, re

video_file_path = "" #@param {type:"string"}
start_time = "00:00:00.000" #@param {type:"string"}
end_time = "00:01:00.000" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -ss "$startTime" -to "$endTime" -c copy "$outputPath"/"$fileName"-TRIM."$fileExtension"

In [ ]:
#@title Extract Audio/Video from File (Lossless)

import os, sys, re

video_file_path = "" #@param {type:"string"}

Extract = 'video_only' #@param ["audio_only","video_only"]
#@markdown For Audio, please choose m4a, mp3, opus, flac, wav file extension.

#@markdown For Video, please choose mkv or mp4 extension.

output_file_extension = 'mkv' #@param ["m4a", "mp3", "opus", "flac", "wav", "mkv", "mp4"]

delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
output_file_path = re.search("^[\/].+\/", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = output_file_extension

if Extract == "audio_only":

 !ffmpeg -hide_banner -i "$inputFile" -vn -c:a copy "$outputPath"/"$fileName"-audio."$fileType"

else:

 !ffmpeg -hide_banner -i "$inputFile" -vcodec copy -an "$outputPath"/"$fileName"-video."$fileType"

In [ ]:
#@title Crop Video
import os, sys, re

video_file_path = "" #@param {type:"string"}
out_width = "1280" #@param {type:"string"}
out_height = "200" #@param {type:"string"}
starting_position_x = "0" #@param {type:"string"}
starting_position_y = "300" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['outWidth'] = out_width
os.environ['outHeight'] = out_height
os.environ['positionX'] = starting_position_x
os.environ['positionY'] = starting_position_y
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -filter:v "crop=$outWidth:$outHeight:$positionX:$positionY" "$outputPath"/"$fileName"-CROP."$fileExtension"

In [ ]:
#@title Extract Individual Frames from Video (*Lossless*)
import os, sys, re

video_file_path = "" #@param {type:"string"}
start_time = "00:00:00.000" #@param {type:"string"}
end_time = "00:01:00.000" #@param {type:"string"}
frame_rate = "23.976" #@param {type:"string"}

#@markdown This will create a folder in the same directory titled "`Extracted Frames`"
#@markdown * [*Example*](https://yuju.pw/y/36pP.png) *of output folder*


output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['frameRate'] = frame_rate
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!mkdir "$outputPath"/"Extracted Frames"
!ffmpeg -hide_banner -i "$inputFile" -ss "$startTime" -to "$endTime" -r "$frameRate"/1 "$outputPath"/"Extracted Frames"/frame%04d.png


In [ ]:
#@markdown <h3>← Verify Tracks for Video</h3>
import os, sys, re
video_file_path = "" #@param {type:"string"}

!mkvmerge -i "$video_file_path"

In [ ]:
#@title Extract Subtitle from Video 
import os, sys, re

video_file_path = "" #@param {type:"string"}
output_file_type = 'idx/sub' #@param ["srt", "ass", "idx/sub"]

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['outputExtension'] = output_file_type
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

if output_file_type == 'srt':
    !ffmpeg -hide_banner -i "$inputFile" "$outputPath"/"$fileName"."$outputExtension"

if output_file_type == 'ass':
    !ffmpeg -hide_banner -i "$inputFile" "$outputPath"/"$fileName"."$outputExtension"

if output_file_type == 'idx/sub':
    !mkvextract "$inputFile" tracks 2:"$outputPath"/"$fileName".idx

In [ ]:
#@title Convert Audio Filetype (*mp3, m4a, ogg, flac, etc.*)
import os, sys, re

audio_file_path = "" #@param {type:"string"}
output_file_type = "mp3" #@param ["mp3", "ogg", "m4a", "opus", "flac", "alac", "wav"]

output_file_path = re.search("^[\/].+\/", audio_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", audio_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = audio_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileExtension'] = output_file_type
os.environ['fileName'] = filename_raw

!ffmpeg -hide_banner -i "$inputFile" "$outputPath"/"$fileName"converted."$fileExtension"

## <img src='https://handbrake.fr/img/logo.png' height="30" alt="RcloneLab"/>***HandBrake***

In [ ]:
#@markdown <h3>← Install Handbrake</h3>

from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/totalleecher/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')

os.system('git clone https://github.com/rokibulislaam/colab-ffmpeg-cuda.git')
os.system('cp -r ./colab-ffmpeg-cuda/bin/. /usr/bin/')
os.system('sudo add-apt-repository ppa:stebbins/handbrake-releases -y')
os.system('sudo apt update -y')
os.system('sudo apt install --install-recommends handbrake-gtk handbrake-cli')

clear_output()
print("Install Finished")

In [ ]:
#@title <center> HandBrake Configuration </center>
#@markdown > Select Mode ( Batch conversion/ Single File)
MODE = "SINGLE" #@param ["SINGLE", "BATCH"]


#@markdown ---
SOURCE = "" #@param {type:"string"}
DESTINATION = "" #@param {type:"string"}
FORMAT = "mp4" #@param ["mp4", "mkv"]
RESOLUTION = "480p" #@param ["360p", "480p","540p", "720p", "1080p"]
Encoder = "x264" #@param ["x264", "x265"]
Encoder_Preset = "ultrafast" #@param ["ultrafast", "faster", "fast", "medium", "slow", "slower"]
#@markdown Choose Constant Quality Rate [Lower = Higher Quality/
#@markdown Larger File Size]
CQ = 30 #@param {type:"slider", min:10, max:30, step:1}
BURN_SUBTITLES = False #@param {type:"boolean"}
Additional_Flags = "" #@param {type:"string"}
#@markdown ---
Email_Alert = False #@param {type:"boolean"}
MESSAGE = "HandBrakeCLI_Colab has finished working" #@param ["HandBrake has finished working"] {allow-input: true}
SENDER_ID ="" #@param {type:"string"}
SENDER_PASS="" #@param {type:"string"}
RECEIVER_ID="" #@param {type:"string"}
#@markdown >#####  Using an app specific password is recommended: https://myaccount.google.com/apppasswords

########################################################
import smtplib
import os


formats = ('.mkv','.mp4','.ts','.avi','.mov','.wmv')

######## Renames the file ########
def fileName(fPath):
        tName = fPath.split('/')[-1]        
        if tName.endswith('ts'):
          tName = '[HANDY] ' + tName[:-3] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'    
        else:
          tName = '[HANDY] ' + tName[:-4] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'    
        return tName

def set_resolution():
  global w,h,flags
  if RESOLUTION == "360p":
    w, h = "480" , "360"
  elif RESOLUTION == "480p":
    w, h = "854" , "480"
  elif RESOLUTION == "540p":
    w, h = "960" , "540"
  elif RESOLUTION == "720p":
    w, h = "1280" , "720"
  elif RESOLUTION == "1080p":
    w, h = "1920" , "1080"

def addFlags():
  global flags
  flags = f" --encoder {Encoder}  --all-audio -s '0,1,2,3' --cfr --optimize --quality={CQ} --width={w} --height={h} --format={FORMAT} --encoder-preset={Encoder_Preset} "
  if BURN_SUBTITLES:
    flags += "-s '1' --subtitle-burned '1' "
  if Additional_Flags != "":
    flags += str(Additional_Flags)

set_resolution()
addFlags()

##### HandBrake and Rclone #####
def runner(path):
  f_name = fileName(path)
  hTemp=f"/content/temp/HandbrakeTemp/{f_name}"
  !HandBrakeCLI -i "$path" -o "$hTemp" $flags
  
      
  if os.path.isfile(hTemp):
    print(f"\n\n********** Successfully converted {f_name}\n Now saving to Destination.....")
    if os.path.exists('/usr/bin/rclone'):
      !rclone move "$hTemp" --user-agent "Mozilla" "$DESTINATION" --transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40
    else:
      dest = DESTINATION+'/'+f_name
      !mv "$hTemp" "$dest"
  if os.path.isfile(DESTINATION+ '/' +f_name):    
      print(f"\n\n********** Successfully saved {f_name} to Destination")

########## Check Mode ########
if MODE=="BATCH":
  os.makedirs(DESTINATION, exist_ok=True)
  if SOURCE.endswith('/'):
      pass
  else: SOURCE +='/'
  filesList = os.listdir(SOURCE+'.')
  if os.path.isfile(SOURCE+'processed_db.txt'):
    pass
  else:
    with open((SOURCE+'processed_db.txt'), 'w') as fb:
      fb.write("Do not delete this file until all files have been processed!\n")
      fb.close()
  with open((SOURCE+'processed_db.txt'), "r+") as filehandle:
    processedList = [x.rstrip() for x in filehandle.readlines()]

    print('<<<<<<<<<<<<<<<<<< Starting Conversion in Batch mode. >>>>>>>>>>>>>>>>>>')

    for currentFile in filesList:
      if currentFile.endswith(formats):
        if currentFile not in processedList:
          currentPath = SOURCE + currentFile      
          print(f'\n\n**************** Current File to process: {currentFile}')
          runner(currentPath)
          filehandle.write(currentFile+'\n')
    filehandle.close()
        

else:
    if SOURCE.endswith(formats):    
        runner(SOURCE)
    else: print("Are you sure you have selected the correct file??")

########### Sending Notification ###############
if Email_Alert == True: 
  from email.mime.multipart import MIMEMultipart 
  from email.mime.text import MIMEText 
  msg = MIMEMultipart()
  msg['From'] = SENDER_ID
  msg['To'] = RECEIVER_ID
  msg['Subject'] = "HandBrakeCLI_Colab has finished executing!" 
  
  msg.attach(MIMEText(MESSAGE))
  s = smtplib.SMTP('smtp.gmail.com', 587) 
  s.starttls() 
  s.login(SENDER_ID, SENDER_PASS)
  s.sendmail(SENDER_ID, RECEIVER_ID, msg.as_string())
  print("Email Alert Sent!")  
  s.quit()


##<img src='https://yuju.pw/y/qUkg.png' height="30" alt="7zip"><i>__7-Zip + UNRAR__</i>

In [ ]:
#@markdown <h3><b>←</b> <i>Required:</i> Install 7-Zip + UNRAR
from IPython.display import clear_output
!sudo apt update
!sudo apt install p7zip-full p7zip-rar unrar rar
clear_output()
print("Install Finished")

In [ ]:
#@markdown <h3>» Run to change current working directory
import os
new_dir = "" #@param {type:"string"}

%cd "$new_dir"

In [ ]:
#@title » Extract Archive (<i>.zip, .rar, .7z, etc.</i>)
import os, sys, re

archive_file_path = "" #@param {type:"string"}
output_path = "" #@param {type:"string"}
os.environ['inputFile'] = archive_file_path
os.environ['outputFile'] = output_path

!7z e "$inputFile"  -o"$outputFile"

In [ ]:
#@title » Create .zip .7z from Folder
import os, sys, re

folder_path = "" #@param {type:"string"}
archiving_method = 'zip' #@param ["zip", "7z" ]

output_file_path = re.search("^[\/].+\/", folder_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", folder_path)
folder_name = re.sub("^[\/]", "", delsplit.group(0))

os.environ['inputDir'] = folder_path
os.environ['outputPath'] = output_file_path_raw
os.environ['folderName'] = folder_name

if archiving_method == 'zip':
   !7z a -tzip "$outputPath"/"$folderName".zip "$inputDir"

if archiving_method == '7z':
   !7z a -t7z -m0=lzma -mx=9 -mfb=258 -md=32m -ms=on "$outputPath"/"$folderName".7z "$inputDir"

##<img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/zip.png' height="30" alt="netdata"> __Extracte/Archive Zip/Tar/Rar__

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Extract Files
MODE = "7Z"  # @param ["UNZIP", "UNTAR", "UNRAR", "7Z"]
PATH_TO_FILE = ""  # @param {type:"string"}
extractPath = ""  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# ================================================================ #
import os, urllib.request
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/totalleecher/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    checkAvailable,
)

def extractFiles():
    global extractPath
    if ARCHIVE_PASSWORD:
      passADD = f'-P {ARCHIVE_PASSWORD}'
    else:
      passADD = ''
    if not extractPath:
      extractPath = "/content/extract"
    os.makedirs(extractPath, exist_ok=True)
    if MODE == "UNZIP":
        runSh('unzip '+passADD+f' "{PATH_TO_FILE}" -d "{extractPath}"', output=True)
    elif MODE == "UNRAR":
        runSh(f'unrar x "{PATH_TO_FILE}" "{extractPath}" '+passADD+' -o+', output=True)
    elif MODE == "UNTAR":
        runSh(f'tar -C "{extractPath}" -xvf "{PATH_TO_FILE}"', output=True)
    else:
        runSh(f'7z x "{PATH_TO_FILE}" -o{extractPath} '+passADD, output=True)


extractFiles()

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Archive Files
MODE = "ZIP" #@param ["ZIP", "TAR", "7Z"]
FILENAME = ""  # @param {type:"string"}
PATH_TO_FILE = ""  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# option supports b k m g (bytes, kilobytes, megabytes, gigabytes)
SPLIT = "no" #@param ["1g", "2g", "3g", "4g", "5g", "no"]


compress = 3#@param  {type:"slider", min:0, max:9, step:0}
#@markdown <font size=2%>Use this to path separate between to this character `|`. Ex `path/to /1 | path/to/2` </br>#Split only in zip mode.(Only for noobs)
# ================================================================ #
from pathlib import PurePosixPath

pathList = PATH_TO_FILE.split('|')
if MODE == "ZIP":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE.ZIP"
    if ARCHIVE_PASSWORD:
      passADD = f'--password "{ARCHIVE_PASSWORD}"'
    else:
      passADD = ''
    splitC = f"-s {SPLIT}" if not 'no' in SPLIT else "" 
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && zip {passADD} -{compress} {splitC} -v -r -u "{FILENAME}" "{partName}"'
      !$cmd
elif MODE == "TAR":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE"
    cmd = f'GZIP=-{compress} tar -zcvf "{FILENAME}.tar.gz" {PATH_TO_FILE}'
    !$cmd
else:
    if not FILENAME:
        FILENAME = "/content/NEW_FILE"
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && 7z a -mx={compress} "{FILENAME}.7z" "{partName}"'
      !$cmd


## <img src='https://rclone.org/img/rclone-32x32.png' height="30" alt="rclone"/><i>__Autorclone__</i>
<h6> ✧  Break The 750GB Limit</h6>


In [ ]:
#@markdown <h3><b>←</b> Run this cell to install all required modules</h3>

!curl https://rclone.org/install.sh | sudo bash
###
%cd /content/
!git clone https://github.com/totalleecher/AutoRclone.git
###
%cd /content/AutoRclone
!sudo pip3 install -r requirements.txt

from IPython.display import HTML, clear_output
clear_output()
print("Successfully Installed")

In [ ]:
#@markdown <h3><b>←</b> Enable Drive Api & Get credentials.json</h3>
!python3 gen_sa_accounts.py --quick-setup -1

from IPython.display import Image, display
display(Image(url = "https://raw.githubusercontent.com/totalleecher/Codemaster/master/Screenshot_2020-03-24%20Python%20Quickstart%20Google%20Drive%20API%20Google%20Developers.png?token=AOP5QOFSDN3WE5Q42J3ANZ26XJ6AA"))

In [ ]:
#@markdown <h3><b>←</b> Upload "credentials.json"</h3>

from google.colab import files
uploaded = files.upload()

In [ ]:
#@markdown <h3><b>←</b> Generate Service Accounts (if you already have service account please skip this step)</h3>

!python3 gen_sa_accounts.py --quick-setup -1

In [ ]:
#@markdown <h3><b>←</b> Existing projects, you need to create a new "Enter the number of the project to be healthy. For example, 5 represents a total of 5 projects and a total of 500 SAs. It is recommended to use googlegroups to manage.</h3>
numbers = "1" #@param {type:"string"}

!python3 gen_sa_accounts.py --quick-setup "$numbers" --new-only

#@markdown <font size="4" color=red><b>If the fourth step has been wrong, you may not have any project. Click here to open <a href="https://console.developers.google.com/" target= "_blank">Google Cloud Platform</a>, press Select a project next to GoogleAPIs in the upper left corner, press NEW PROJECT in the upper right corner of the pop-up window. Then press CREATE. Go back and run the fourth step. </b></font>

### Method 1 (Adding Services Acc to **Team Drive**)

In [ ]:
#@markdown <h3><b>←</b> Run this cell to add all SAs to a Team Drive</h3>
#@markdown </h5>Team Drive ID (TDid) = https://drive.google.com/drive/folders/<code>all-of-these-letters-numbers-and-symbols </code></h5>
#@markdown <h5><i>This may take some time. Be patient!</i></h5> 
import os
TDid = "" #@param {type:"string"}
!python3 add_to_team_drive.py -d "$TDid"

In [ ]:
#@markdown <h3><b>←</b> **Final Step -** Running Autorclone</h3>
#@markdown <p>- Add the Source Folder ID and then the Destination Folder ID</p>
#@markdown <p>- Note: Your source*** *(*`src`*)* ***folder must be publically accessible***</p>

#@markdown Folder IDs are found at the end of the URL:
#@markdown ![](https://yuju.pw/y/qNvy.png)

#@title <h3><b>←</b> Run to clone all folders + files from <code>src</code> to <code>dest</code>
#@markdown <i>Paste only the Folder IDs for these values</i>:
source = "" #@param {type:"string"}
destination = "" #@param {type:"string"}
sa = "" #@param {type:"string"}

!python3 rclone_sa_magic.py -s "$source" -d "$destination" -b "$sa" -e 600

#@markdown <b> Tip - sa start default 1 If you have copied 7.5T, then you should put 11 SA 750GB in 24 hours.

### Method 2 (Adding Services Acc to **Google Group**)

In [ ]:
#@markdown Create Google Group then add the service accounts as members by hand. Limit is 10 at a time, 100 a day but if you read our warning and notes above, you would have 1 project and hence easily in your range.

import os
from IPython.display import HTML, clear_output

print ("https://groups.google.com/")

In [ ]:
#@markdown <h3><b>←</b> Generate Service Accounts (if you already have service account please skip this step)</h3>

!python3 gen_sa_accounts.py --quick-setup -1

In [ ]:
#@markdown <h3><b>←</b> **Final Step -** Running Autorclone</h3>
#@markdown <p>- Add the Source Folder ID and then the Destination Folder ID</p>
#@markdown <p>- Note: ***Your source*** *(*`src`*)* ***folder must be publically accessible***</p>

#@markdown Folder IDs are found at the end of the URL:
#@markdown ![](https://yuju.pw/y/qNvy.png)

#@title <h3><b>←</b> Run to clone all folders + files from <code>src</code> to <code>dest</code>
#@markdown <i>Paste only the Folder IDs for these values</i>:
source = "" #@param {type:"string"}
destination = "" #@param {type:"string"}

!python3 rclone_sa_magic.py -s "$source" -d "$destination" -b 1 -e 600

### Extra (Checking File size & Qty)

In [ ]:
#@markdown <h3><b>←</b> Size of Source Directory</h3> 
!rclone --config rclone.conf size --disable ListR src001:

In [ ]:
#@markdown <h3><b>←</b> Size of Destination Directory</h3> 
!rclone --config rclone.conf size --disable ListR src001:

In [ ]:
#@markdown <h3><b>←</b> Deduplicate the target file after transfer</h3>
!rclone --config rclone.conf dedupe dst001:

###<font size=4 color="red"><b><center>When you first run it, please back up SA(backup) at the end, next time you only need to configure the environment in step 1 when copying, 3(before Download the downloaded credentials.json) Restore Services Account to the configuration environment and start copying</center></b></font>

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive logo"/></center>
#@markdown <center><h3>Load Google Drive backup and restore the Accounts folder under Autoclone. </h3></center><br>

from google.colab import drive
drive.mount('/content/drive')

MODE = "BACKUP" #@param ["BACKUP", "RESTORE"]
#备份SA!
if MODE == "BACKUP": 
  !rclone copy "/content/AutoRclone/accounts/" "/content/drive/My Drive/accounts/"
elif MODE == "RESTORE":
  !rclone copy "/content/drive/My Drive/accounts/" "/content/AutoRclone/accounts/"

# <center><h3>Copyright © 2021. All right reserved.</h3><h5>Source Code - <b>biplobsd, blackpearl, dreamtime, xyou365, NVLabs, DeOldify,  AlphaAtlas, mariosemes, chervonij, idealo, tsurumeso, sniklaus, mrm8488, KMH, rokibulislaam, SKGHD</b><div>Modified by <b>Codemaster@BSSG</b></div><h5>All in one downloader ver 3.0.0</h5></center></h5>